In [ ]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox
import json

# Archivo de configuración para rutas por defecto
config_file = 'rutas_config.json'

# Cargar rutas por defecto desde archivo, si existe
if os.path.exists(config_file):
    with open(config_file, 'r') as f:
        config = json.load(f)
    default_ruta = config.get('ruta', r"C:\Users\amvillamizar\Downloads\Avianca\Offline\Documentos\Publicacion tarifas\Pricing_checks\Formato_airnguru\Formatos\1401_MATCH_5")
    default_excel = config.get('ruta_excel', r"C:\Users\amvillamizar\OneDrive - Aerovias del Continente Americano S.A. AVIANCA\Archivos de Juliana Marin - Pricing Estratégico\Offline\Documents\Archivo validaciones\Parametros Offline_V6.xlsx")
else:
    default_ruta = r"C:\Users\amvillamizar\Downloads\Avianca\Offline\Documentos\Publicacion tarifas\Pricing_checks\Formato_airnguru\Formatos\1401_MATCH_5"
    default_excel = r"C:\Users\amvillamizar\OneDrive - Aerovias del Continente Americano S.A. AVIANCA\Archivos de Juliana Marin - Pricing Estratégico\Offline\Documents\Archivo validaciones\Parametros Offline_V6.xlsx"

# Crear root para topmost
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

# Pregunta inicial: ¿Quieres cambiar las rutas?
cambiar_rutas = messagebox.askyesno("Configuración de rutas", "¿Quieres cambiar las rutas de archivos?")

if cambiar_rutas:
    # Mostrar rutas actuales
    messagebox.showinfo("Rutas actuales", f"Ruta para formatos: {default_ruta}\nRuta para archivo parámetros: {default_excel}")

    # Preguntar por carpeta
    cambiar_ruta = messagebox.askyesno("Cambiar carpeta de formatos", "¿Quieres cambiar la carpeta de formatos?")
    if cambiar_ruta:
        ruta = filedialog.askdirectory(title="Selecciona la carpeta de formatos", initialdir=default_ruta)
        if not ruta:
            ruta = default_ruta
            messagebox.showinfo("Información", "Usando ruta por defecto.")
        else:
            messagebox.showinfo("Información", f"Ruta seleccionada: {ruta}")
    else:
        ruta = default_ruta
        messagebox.showinfo("Información", "Usando ruta por defecto.")

    # Preguntar por archivo
    cambiar_excel = messagebox.askyesno("Cambiar archivo de parámetros", "¿Quieres cambiar el archivo de parámetros?")
    if cambiar_excel:
        ruta_excel = filedialog.askopenfilename(title="Selecciona el archivo de parámetros", initialdir=os.path.dirname(default_excel), filetypes=[("Excel files", "*.xlsx")])
        if not ruta_excel:
            ruta_excel = default_excel
            messagebox.showinfo("Información", "Usando archivo por defecto.")
        else:
            messagebox.showinfo("Información", f"Archivo seleccionado: {ruta_excel}")
    else:
        ruta_excel = default_excel
        messagebox.showinfo("Información", "Usando archivo por defecto.")
else:
    # Usar rutas por defecto sin más preguntas
    ruta = default_ruta
    ruta_excel = default_excel
    messagebox.showinfo("Información", "Usando rutas por defecto.")

messagebox.showinfo("Rutas finales", f"Ruta final: {ruta}\nExcel final: {ruta_excel}")

# Guardar las rutas seleccionadas como nuevas por defecto
config = {'ruta': ruta, 'ruta_excel': ruta_excel}
with open(config_file, 'w') as f:
    json.dump(config, f)
messagebox.showinfo("Información", "Rutas guardadas como nuevas por defecto.")

'ok'

Lectura archivo airnguru

In [2]:
import pandas as pd
import glob
import os
from collections import OrderedDict
import numpy as np
from pathlib import Path
from datetime import datetime
from tabulate import tabulate
import tkinter as tk
from tkinter import filedialog
from openpyxl import load_workbook
from tkinter import simpledialog, messagebox

In [3]:
fecha = datetime.now().strftime("%d%m%y")

In [4]:


# =========================================================
# 1) Ventana emergente para escoger tipo de input (Excel/CSV)
# =========================================================
def elegir_tipo_input_popup() -> str:
    """
    Devuelve 'excel' o 'csv' según selección del usuario en una ventana emergente.
    """
    root = tk.Tk()
    root.title("Selecciona el tipo de archivos")
    root.geometry("320x140")
    root.resizable(False, False)

    var = tk.StringVar(value="excel")  # default

    tk.Label(root, text="¿Qué tipo de archivos hay en la carpeta?", font=("Segoe UI", 10)).pack(pady=10)
    tk.Radiobutton(root, text="Formato AG", variable=var, value="excel").pack(anchor="w", padx=30)
    tk.Radiobutton(root, text="Formato POPS", variable=var, value="csv").pack(anchor="w", padx=30)

    def confirmar():
        root.quit()

    tk.Button(root, text="OK", width=10, command=confirmar).pack(pady=12)

    # Hacerla topmost si se puede
    try:
        root.attributes("-topmost", True)
    except Exception:
        pass

    root.mainloop()
    sel = var.get()
    root.destroy()
    return sel


# =========================================================
# 2) Función auxiliar: lectura robusta de CSV con separador
# =========================================================
def read_csv_sin_headers(path: str) -> pd.DataFrame:
    """
    Lee un CSV sin headers, eliminando las primeras 11 filas.
    Intenta detectar separador (tab, coma, punto y coma).
    Elimina la primera columna (la del ejemplo 'F') y asigna nombres.
    Luego mapea a nombres "tipo Excel" para que tu renombrado funcione.
    """
    seps = ["\t", ",", ";"]

    df = None
    last_err = None

    for sep in seps:
        try:
            tmp = pd.read_csv(
                path,
                header=None,
                skiprows=11,     
                sep=sep,
                engine="python",
                dtype=str,      
                encoding_errors="ignore",
            )
            
            if tmp.shape[1] <= 1:
                continue
            df = tmp
            break
        except Exception as e:
            last_err = e

    if df is None:
        raise ValueError(f"No pude leer el CSV '{path}'. Último error: {last_err}")

    df = df.drop(columns=[0])

    # Tomar solo las primeras 11 columnas esperadas (por si vienen columnas extra)
    # Esperadas después de drop: 11 columnas
    if df.shape[1] < 11:
        raise ValueError(
            f"El CSV '{path}' tiene menos de 11 columnas después de eliminar la primera. "
            f"Columnas detectadas: {df.shape[1]}"
        )
    df = df.iloc[:, :11]

    # Asignar nombres (porque no hay headers)
    csv_cols = [
        "CARRIER_CODE",
        "FARE_TARIFF_CODE",
        "USER_ACTION_CODE",
        "ORIGIN_AIRPORT_CITY_CODE",
        "DESTINATION_AIRPORT_CITY_CODE",
        "FARE_CLASS_CODE",
        "OW_RT_IND",
        "ROUTING_NUMBER",
        "FOOTNOTE_CODE",
        "CURRENCY_CODE",
        "FARE_AMOUNT",
    ]
    df.columns = csv_cols

    return df


# =========================================================
# 3) Funciones que "empatan" lógica para Excel/CSV
# =========================================================
def obtener_columnas_archivo(path: str, tipo: str, sheet=0):
    """
    Para tu "primera pasada": obtener columnas sin leer todo el archivo.
    """
    if tipo == "excel":
        cols = pd.read_excel(path, sheet_name=sheet, nrows=0).columns
        cols = pd.Index([str(c).strip() for c in cols])
        return cols

    elif tipo == "csv":
        return pd.Index([
            "CARRIER_CODE",
            "FARE_TARIFF_CODE",
            "USER_ACTION_CODE",
            "ORIGIN_AIRPORT_CITY_CODE",
            "DESTINATION_AIRPORT_CITY_CODE",
            "FARE_CLASS_CODE",
            "OW_RT_IND",
            "ROUTING_NUMBER",
            "FOOTNOTE_CODE",
            "CURRENCY_CODE",
            "FARE_AMOUNT",
        ])
    else:
        raise ValueError("tipo debe ser 'excel' o 'csv'")


def leer_archivo(path: str, tipo: str, sheet=0) -> pd.DataFrame:
    """
    Para tu "segunda pasada": leer el archivo completo ya listo para tu pipeline.
    """
    if tipo == "excel":
        df = pd.read_excel(path, sheet_name=sheet)
        df.columns = pd.Index([str(c).strip() for c in df.columns])
        return df

    elif tipo == "csv":
        df = read_csv_sin_headers(path)
        # ya viene con nombres "tipo Excel", solo normalizamos strings
        df.columns = pd.Index([str(c).strip() for c in df.columns])
        return df

    else:
        raise ValueError("tipo debe ser 'Formato AG' o 'Formato POPS'")




## Poner ruta del formato input y ruta donde se guarda el resumen

In [5]:
# ========================================================
def write_sheet(df, path, sheet_name, replace=True):
    path = Path(path)
    if not path.parent.exists():
        path.parent.mkdir(parents=True, exist_ok=True)

    if path.exists():
        mode = "a"
        if_sheet_exists = "replace" if replace else "overlay"
    else:
        mode = "w"
        if_sheet_exists = None

    with pd.ExcelWriter(path, engine="openpyxl", mode=mode,
                        if_sheet_exists=if_sheet_exists) as writer:
        df.to_excel(writer, index=False, sheet_name=sheet_name)


# ========================================================
# MAIN: Selección tipo + ruta + glob + pipeline
# ========================================================

tipo_input = elegir_tipo_input_popup()
print(f"✅ Tipo seleccionado: {tipo_input}")

patron = "*.xlsx" if tipo_input == "excel" else "*.csv"
archivos = glob.glob(os.path.join(ruta, patron))
print("Archivos encontrados:")
print(archivos)

if not archivos:
    print(f"❌ No se encontraron archivos con patrón {patron} en: {ruta}")
    df_descarga = pd.DataFrame()
else:
    root = tk.Tk()
    root.withdraw()
    try:
        root.attributes("-topmost", True)
    except Exception:
        pass

    excel_path = filedialog.asksaveasfilename(
        title="Guardar archivo Excel",
        initialfile=f"{fecha}_Resumen.xlsx" if 'fecha' in globals() else "Resumen.xlsx",
        defaultextension=".xlsx",
        filetypes=[("Excel (*.xlsx)", "*.xlsx")]
    )

    if not excel_path:
        print("❌ Operación cancelada por el usuario. No se exportará nada.")
        df_descarga = pd.DataFrame()
    else:
        excel_path = Path(excel_path)
        excel_path.parent.mkdir(parents=True, exist_ok=True)
        reporte_path = excel_path.with_suffix(".txt")

        out_dir = Path(r"C:\Users\amvillamizar\Downloads\Avianca\Offline\Documentos\Publicacion tarifas\Pricing_checks\Parametros_Offline\Resumen")
        out_dir.mkdir(parents=True, exist_ok=True)
        excel_path = out_dir / (Path(excel_path).name)
        reporte_path = excel_path.with_suffix(".txt")


        SHEET = 0

        cols_por_archivo = []
        orden_global = OrderedDict()

        for f in archivos:
            cols = obtener_columnas_archivo(f, tipo_input, sheet=SHEET)
            cols_por_archivo.append({"archivo": os.path.basename(f), "path": f, "cols": cols})

            for c in cols:
                if c not in orden_global:
                    orden_global[c] = None

        todas_las_columnas = list(orden_global.keys())

        dfs = []
        resumen = []

        for item in cols_por_archivo:
            f_name = item["archivo"]
            ruta_archivo = item["path"]

            df = leer_archivo(ruta_archivo, tipo_input, sheet=SHEET)

            faltan = [c for c in todas_las_columnas if c not in df.columns]
            presentes = list(df.columns)

            for c in faltan:
                df[c] = pd.NA

            df = df[todas_las_columnas]

            dfs.append(df)
            resumen.append({
                "archivo": f_name,
                "presentes": presentes,
                "faltan_vs_total": faltan
            })

        print("=== Control de columnas por archivo (vs. universo total) ===")
        for r in resumen:
            print(f"- {r['archivo']}:")
            print(f"   • Presentes: {len(r['presentes'])}")
            print(f"   • Faltan: {len(r['faltan_vs_total'])} -> {r['faltan_vs_total']}")

        print("\nTotal de columnas en el universo:", len(todas_las_columnas))
        print("Listado completo (orden de primera aparición):")
        print(todas_las_columnas)

        if dfs:
            df_descarga = pd.concat(dfs, ignore_index=True)
            print("\nUnión lista -> df_descarga.shape:", df_descarga.shape)

            renombres = {
                "ORIGIN_AIRPORT_CITY_CODE": "Origin",
                "DESTINATION_AIRPORT_CITY_CODE": "Destination",
                "CARRIER_CODE": "Carrier",
                "FARE_TARIFF_NUMBER": "Tariff number",
                "FARE_TARIFF_CODE": "Tariff code",
                "FARE_CLASS_CODE": "Fareclass",
                "CURRENCY_CODE": "Currency",
                "FARE_AMOUNT": "Base price",
                "EFFDATE": "Effective date",
                "OW_RT_IND": "OW/RT",
                "ROUTING_NUMBER": "Routing number",
                "FOOTNOTE_CODE": "FN",
                "USER_ACTION_CODE": "Action code"
            }

            df_descarga = df_descarga.rename(columns=renombres)

            print("Columnas después de renombrar:")
            print(df_descarga.columns.tolist())

            write_sheet(df_descarga, excel_path, sheet_name="Resumen", replace=True)
            print(f"✅ Excel generado en: {excel_path}")
            print(f"ℹ️ (Si luego quieres) Reporte txt sería: {reporte_path}")
        else:
            df_descarga = pd.DataFrame()
            print("\nNo hay dataframes para concatenar.")

✅ Tipo seleccionado: excel
Archivos encontrados:
['C:/Users/amvillamizar/Downloads/Avianca/Offline/Documentos/Publicacion tarifas/Pricing_checks/Formato_airnguru/Formatos/1401_MATCH_UA_3\\1401_MATCH_UA_3.xlsx', 'C:/Users/amvillamizar/Downloads/Avianca/Offline/Documentos/Publicacion tarifas/Pricing_checks/Formato_airnguru/Formatos/1401_MATCH_UA_3\\smf.xlsx']
=== Control de columnas por archivo (vs. universo total) ===
- 1401_MATCH_UA_3.xlsx:
   • Presentes: 23
   • Faltan: 0 -> []
- smf.xlsx:
   • Presentes: 23
   • Faltan: 0 -> []

Total de columnas en el universo: 23
Listado completo (orden de primera aparición):
['ORIGIN_AIRPORT_CITY_CODE', 'DESTINATION_AIRPORT_CITY_CODE', 'CARRIER_CODE', 'FARE_TARIFF_NUMBER', 'FARE_TARIFF_CODE', 'FARE_CLASS_CODE', 'CURRENCY_CODE', 'FARE_AMOUNT', 'OW_RT_IND', 'ROUTING_NUMBER', 'RULE_NUMBER', 'FOOTNOTE_CODE', 'USER_ACTION_CODE', 'FIRST_TICKETING_STR', 'LAST_TICKETING_STR', 'TRAVEL_START_STR', 'TRAVEL_START_TO_STR', 'TRAVEL_END_STR', 'FARE_NOTE', 'CHAN

In [6]:
# Diccionario: {"columna_original": "nuevo_nombre"}
renombres = {
    "ORIGIN_AIRPORT_CITY_CODE": "Origin",
    "DESTINATION_AIRPORT_CITY_CODE": "Destination",
    "CARRIER_CODE": "Carrier",
    "FARE_TARIFF_NUMBER": "Tariff number",
    "FARE_TARIFF_CODE": "Tariff code",
    "FARE_CLASS_CODE": "Fareclass",
    "CURRENCY_CODE": "Currency",
    "FARE_AMOUNT": "Base price",
    "EFFDATE": "Effective date",
    "OW_RT_IND":"OW/RT",
    "ROUTING_NUMBER":"Routing number",
    'FOOTNOTE_CODE':"FN",
    'USER_ACTION_CODE':"Action code"

    }

# Aplicar renombrado en df_descarga
df_descarga = df_descarga.rename(columns=renombres)

print("Columnas después de renombrar:")
print(df_descarga.columns.tolist())

Columnas después de renombrar:
['Origin', 'Destination', 'Carrier', 'Tariff number', 'Tariff code', 'Fareclass', 'Currency', 'Base price', 'OW/RT', 'Routing number', 'RULE_NUMBER', 'FN', 'Action code', 'FIRST_TICKETING_STR', 'LAST_TICKETING_STR', 'TRAVEL_START_STR', 'TRAVEL_START_TO_STR', 'TRAVEL_END_STR', 'FARE_NOTE', 'CHANGE CURRENCY', 'CHANGE FN', 'CHANGE RTG', 'EFF DATE']


In [7]:
#print(df_descarga["Currency"].unique())
#print(df_descarga["Currency"].value_counts())
#print(df_descarga["Routing number"].unique())
##print(df_descarga["Routing number"].value_counts())

In [8]:
df_descarga["OD"] = df_descarga["Origin"].astype(str) + df_descarga["Destination"].astype(str)

In [9]:
df_descarga["Tariff code"].value_counts() 

Series([], Name: count, dtype: int64)

Cambiar columna RT a OW

In [10]:
# 1. Crear copia de la columna
df_descarga["Base price_OW"] = df_descarga["Base price"]

# 2. Renombrar: la copia toma el nombre original, la original toma uno nuevo
df_descarga.rename(columns={"Base price_OW": "Base price", "Base price": "Base price_inicial"}, inplace=True)

print(df_descarga.head(10))

  Origin Destination Carrier  Tariff number  Tariff code Fareclass Currency  \
0    UIO         MSP      AV            NaN          NaN  VLRB1BRS      USD   
1    UIO         MSP      AV            NaN          NaN  VLRB1BRS      USD   
2    UIO         MSP      AV            NaN          NaN  VLRB2BRS      USD   
3    UIO         MSP      AV            NaN          NaN  VLRB2BRS      USD   
4    UIO         MSP      AV            NaN          NaN  VLRB3BRS      USD   
5    UIO         MSP      AV            NaN          NaN  VLRB3BRS      USD   
6    UIO         MSP      AV            NaN          NaN  WHMB1BRS      USD   
7    UIO         MSP      AV            NaN          NaN  WHMB1BRS      USD   
8    UIO         MSP      AV            NaN          NaN  WHMB2BRS      USD   
9    UIO         MSP      AV            NaN          NaN  WHMB2BRS      USD   

   Base price_inicial  OW/RT  Routing number  ...  TRAVEL_START_STR  \
0                2274      2             414  ...          

In [11]:
# Asegurar que ambas columnas sean numéricas (por si vienen como texto desde CSV)
df_descarga["Base price"] = pd.to_numeric(df_descarga["Base price"], errors="coerce")
df_descarga["Base price_inicial"] = pd.to_numeric(df_descarga["Base price_inicial"], errors="coerce")

# (opcional pero recomendado) asegurar OW/RT numérico también
df_descarga["OW/RT"] = pd.to_numeric(df_descarga["OW/RT"], errors="coerce")

df_descarga.loc[df_descarga["OW/RT"] == 2, "Base price"] = df_descarga["Base price_inicial"] / 2


df_descarga.head(5)

C:\Users\amvillamizar\AppData\Local\Temp\ipykernel_12532\3681056995.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1137. 1137. 1227. ...  205.  255.  350.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_descarga.loc[df_descarga["OW/RT"] == 2, "Base price"] = df_descarga["Base price_inicial"] / 2


,Origin,Destination,Carrier,Tariff number,Tariff code,Fareclass,Currency,Base price_inicial,OW/RT,Routing number,...,TRAVEL_START_STR,TRAVEL_START_TO_STR,TRAVEL_END_STR,FARE_NOTE,CHANGE CURRENCY,CHANGE FN,CHANGE RTG,EFF DATE,OD,Base price
0,UIO,MSP,AV,NaN,NaN,VLRB1BRS,USD,2274,2,414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UIOMSP,1137.0
1,UIO,MSP,AV,NaN,NaN,VLRB1BRS,USD,2274,2,418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UIOMSP,1137.0
2,UIO,MSP,AV,NaN,NaN,VLRB2BRS,USD,2454,2,414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UIOMSP,1227.0
3,UIO,MSP,AV,NaN,NaN,VLRB2BRS,USD,2454,2,418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UIOMSP,1227.0
4,UIO,MSP,AV,NaN,NaN,VLRB3BRS,USD,2704,2,414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UIOMSP,1352.0


In [12]:
# === 1) Diccionario de tasas -> USD ===
rates_usd = {
    "GBP": 1.3035391, "EUR": 1.16486165, "USD": 1, "DZD": 0.00769762, "AWG": 0.55555556,
    "AED": 0.27235053, "CNY": 0.13794692, "INR": 0.01198647, "CHF": 1.11187422,
    "HUF": 0.00276633, "EGP": 0.0210084, "DKK": 0.14441221, "QAR": 0.27472527,
    "SEK": 0.09364955, "NOK": 0.09968775, "CZK": 0.04354228, "MAD": 0.09910403,
    "TWD": 0.0307374, "PLN": 0.25208611, "CAD": 0.70838835, "JOD": 1.41164965,
    "BHD": 2.65251662, "AUD": 0.66213709, "SAR": 0.26666667, "SGD": 0.73602151,
    "GIP": 1.27218544,"ANG": 0.55865922
}


In [13]:

# === 2) Normaliza campos ===
col_currency = "Currency"
col_tarifa = "Base price"

df_descarga[col_tarifa] = (
    df_descarga[col_tarifa]
      .astype(str)
      .str.replace(",", "", regex=False)
      .str.replace(r"\s+", "", regex=True)
      .str.replace("$", "", regex=False)
      .str.replace("USD", "", regex=False)
      .str.strip()
)
df_descarga[col_tarifa] = pd.to_numeric(df_descarga[col_tarifa], errors="coerce")

df_descarga[col_currency] = df_descarga[col_currency].astype(str).str.upper().str.strip()

# === 3) Aplica tasa y crea columna en USD ===
df_descarga["Rate_USD"] = df_descarga[col_currency].map(rates_usd)

faltantes = sorted(set(df_descarga.loc[df_descarga["Rate_USD"].isna(), col_currency]))
if faltantes:
    print("Monedas sin tasa:", faltantes)

df_descarga["Tarifa_USD"] = df_descarga[col_tarifa] * df_descarga["Rate_USD"]
df_descarga["Tarifa_USD"]=df_descarga["Tarifa_USD"].round(1)

df_descarga["Tarifa_inicial_USD"] = df_descarga["Base price_inicial"] * df_descarga["Rate_USD"]
df_descarga["Tarifa_inicial_USD"]=df_descarga["Tarifa_inicial_USD"].round(1)


In [14]:
#Crear columna de Cabin
df_descarga["Cabin"] = np.where(
    df_descarga["Fareclass"].astype(str).str[0].isin(["F", "D", "J", "C"]),
    "C",  
    "Y"
)

print(df_descarga[["Fareclass", "Cabin"]].head())

  Fareclass Cabin
0  VLRB1BRS     Y
1  VLRB1BRS     Y
2  VLRB2BRS     Y
3  VLRB2BRS     Y
4  VLRB3BRS     Y


In [15]:
## Crear columnas de RBD y Bundle
df_descarga["RBD"]=df_descarga["Fareclass"].astype(str).str[0]
df_descarga["Bundle"]=df_descarga["Fareclass"].astype(str).str[3:5]

In [16]:
df_descarga.head(5)

,Origin,Destination,Carrier,Tariff number,Tariff code,Fareclass,Currency,Base price_inicial,OW/RT,Routing number,...,CHANGE RTG,EFF DATE,OD,Base price,Rate_USD,Tarifa_USD,Tarifa_inicial_USD,Cabin,RBD,Bundle
0,UIO,MSP,AV,NaN,NaN,VLRB1BRS,USD,2274,2,414,...,NaN,NaN,UIOMSP,1137.0,1.0,1137.0,2274.0,Y,V,B1
1,UIO,MSP,AV,NaN,NaN,VLRB1BRS,USD,2274,2,418,...,NaN,NaN,UIOMSP,1137.0,1.0,1137.0,2274.0,Y,V,B1
2,UIO,MSP,AV,NaN,NaN,VLRB2BRS,USD,2454,2,414,...,NaN,NaN,UIOMSP,1227.0,1.0,1227.0,2454.0,Y,V,B2
3,UIO,MSP,AV,NaN,NaN,VLRB2BRS,USD,2454,2,418,...,NaN,NaN,UIOMSP,1227.0,1.0,1227.0,2454.0,Y,V,B2
4,UIO,MSP,AV,NaN,NaN,VLRB3BRS,USD,2704,2,414,...,NaN,NaN,UIOMSP,1352.0,1.0,1352.0,2704.0,Y,V,B3


In [17]:
cols_wanted = [
    "Tariff code",
    "Origin",
    "Destination",
    "Fareclass",
    "OW/RT",
    "Routing number",
    "Action code",
    "FN",
    "Currency",
    "Base price",
    "Tarifa_USD",
    "RBD",
    "OD",
    "Cabin",
    "Bundle",
    "Tarifa_inicial_USD"

]
# 2) Opción robusta: toma solo las que existan y avisa cuáles faltan
existentes = [c for c in cols_wanted if c in df_descarga.columns]
faltantes  = [c for c in cols_wanted if c not in df_descarga.columns]
df_filtrado_final = df_descarga.loc[:, existentes]
if faltantes:
    print("⚠️ No se encontraron estas columnas:", faltantes)

## Validacion corners

In [18]:
df_filtrado_final.head(10)

,Tariff code,Origin,Destination,Fareclass,OW/RT,Routing number,Action code,FN,Currency,Base price,Tarifa_USD,RBD,OD,Cabin,Bundle,Tarifa_inicial_USD
0,NaN,UIO,MSP,VLRB1BRS,2,414,NaN,NaN,USD,1137.0,1137.0,V,UIOMSP,Y,B1,2274.0
1,NaN,UIO,MSP,VLRB1BRS,2,418,NaN,NaN,USD,1137.0,1137.0,V,UIOMSP,Y,B1,2274.0
2,NaN,UIO,MSP,VLRB2BRS,2,414,NaN,NaN,USD,1227.0,1227.0,V,UIOMSP,Y,B2,2454.0
3,NaN,UIO,MSP,VLRB2BRS,2,418,NaN,NaN,USD,1227.0,1227.0,V,UIOMSP,Y,B2,2454.0
4,NaN,UIO,MSP,VLRB3BRS,2,414,NaN,NaN,USD,1352.0,1352.0,V,UIOMSP,Y,B3,2704.0
5,NaN,UIO,MSP,VLRB3BRS,2,418,NaN,NaN,USD,1352.0,1352.0,V,UIOMSP,Y,B3,2704.0
6,NaN,UIO,MSP,WHMB1BRS,1,414,NaN,NaN,USD,322.0,322.0,W,UIOMSP,Y,B1,322.0
7,NaN,UIO,MSP,WHMB1BRS,1,418,NaN,NaN,USD,322.0,322.0,W,UIOMSP,Y,B1,322.0
8,NaN,UIO,MSP,WHMB2BRS,1,414,NaN,NaN,USD,412.0,412.0,W,UIOMSP,Y,B2,412.0
9,NaN,UIO,MSP,WHMB2BRS,1,418,NaN,NaN,USD,412.0,412.0,W,UIOMSP,Y,B2,412.0


In [19]:
# Limpiar Fareclass
df_filtrado_final["Fareclass"] = df_filtrado_final["Fareclass"].astype(str).str.strip()

# Reglas
cond1 = (df_filtrado_final["Fareclass"].str[0].isin(["U", "T", "S"])) & (df_filtrado_final["Fareclass"].str[3:5] == "B3")
cond2 = (df_filtrado_final["Fareclass"].str[0].isin(["H", "M", "B", "Y"])) & (df_filtrado_final["Fareclass"].str[3:5].isin(["B0", "B1"]))
cond3 = (df_filtrado_final["Fareclass"].str[0] == "F") & (df_filtrado_final["Fareclass"].str[3:5] == "B5")
cond4 = (df_filtrado_final["Fareclass"].str[0] == "C") & (df_filtrado_final["Fareclass"].str[3:5] == "B4")

# Filtrar todas las violaciones
violaciones = df_filtrado_final[cond1 | cond2 | cond3 | cond4]

print(f"Número de filas que incumplen reglas: {len(violaciones)}")
print(violaciones[["Fareclass"]].head(20))


Número de filas que incumplen reglas: 0
Empty DataFrame
Columns: [Fareclass]
Index: []


## Bundle minimo

In [20]:
# --- Config ---
COL_OD = "OD"
COL_BUNDLE = "Bundle"
niveles = [f"B{i}" for i in range(6)]  # B0..B5
# --- Nueva config ---
COL_RBD = "RBD"                      
RBD_SOLO_B5 = {"F", "D", "J", "C"}  # conjunto permitido para forzar B5


# Endpoints “permitidos” para que el objetivo sea B0 (si ambos pertenecen)
ENDPOINTS = {
    "ADZ","BAQ","BGA","BOG","CLO","CTG","CUC","MDE","PEI","PTY","PUJ","SJU",
    "BCN","MAD","PAR","MEX","YMQ","YTO","BOS","CHI","FLL","MCO","MIA","NYC","WAS",
    "AUA","SJO","CUR","SDQ","SAL","GYE","UIO","GUA","CUN","EJA","MTR","PSO","RCH",
    "SMR","VUP","CUZ","LIM","AXM","EYP","IBE","IPI","NVA","PPN","VVC","LON","MAO",
    "ASU","MVD","BUE","TUY","UIB","LET","LPB","VVI","SCL","SAO","GEO","RIO","SAP",
    "XPL","CUE","MEC","EZE","COR"
}

# Excepciones que fuerzan objetivo B1
CITY_FORCE_B1 = {"RIO","SAO","MAO","BEL","MAD","BCN","LON","PAR"}
CITY_FORCE_B2={"CCS"}     # si ORIG o DEST es alguno de estos
OD_FORCE_B1   = {"LAXSAL"}               # ODs completos que fuerzan B1

# --- Normalización y filtrado de valores válidos ---
s = df_filtrado_final[COL_BUNDLE].astype(str).str.strip().str.upper()
valid = s.isin(niveles)
dfb = df_filtrado_final.loc[valid, [COL_OD, COL_BUNDLE]].copy()
if dfb.empty:
    raise ValueError("No hay valores válidos en la columna 'Bundle' (esperados: B0..B5).")

dtype = pd.CategoricalDtype(categories=niveles, ordered=True)
dfb[COL_BUNDLE] = dfb[COL_BUNDLE].astype(dtype)

# --- 1) Mínimo por OD ---
min_por_od_cat = dfb.groupby(COL_OD, dropna=False)[COL_BUNDLE].min()
min_por_od = min_por_od_cat.astype(str)

# --- 2) Objetivo por OD según reglas ---
def objetivo_por_od(od: str) -> str:
    if pd.isna(od) or len(str(od)) < 6:
        return "B1"  # conservador
    od = str(od).upper().strip()
    if od in OD_FORCE_B1:
        return "B1"
    orig, dest = od[:3], od[3:]
        # NUEVA excepción: si ORIG o DEST es CCS -> B2
    if orig in CITY_FORCE_B2 or dest in CITY_FORCE_B2:
        return "B2"
    # Si alguno está en la lista que fuerza B1
    if orig in CITY_FORCE_B1 or dest in CITY_FORCE_B1:
        return "B1"
    # Si ambos endpoints están en ENDPOINTS -> B0; de lo contrario B1
    return "B0" if (orig in ENDPOINTS and dest in ENDPOINTS) else "B1"

objetivo_series = min_por_od.index.to_series().map(objetivo_por_od)
objetivo_series.index = min_por_od.index
objetivo_series.name = "Bundle_objetivo"

# --- 2.5) Regla extra: si para un OD SOLO hay RBD en {A,C,BC,D} -> objetivo = B5 ---
# 1) Conjunto de RBD por OD (ignorando NaN y espacios)
rbd_por_od = (df_filtrado_final
              .assign(_rbd=df_filtrado_final[COL_RBD].astype(str).str.strip().str.upper())
              .groupby(COL_OD, dropna=False)["_rbd"]
              .agg(lambda s: {x for x in s if x not in {"", "NAN"}}))

# 2) Marca los OD cuyo conjunto de RBD es no vacío y SUBCONJUNTO de {A,C,BC,D}
solo_rbd_acbcd = rbd_por_od.map(lambda s: bool(s) and s.issubset(RBD_SOLO_B5))

# 3) Aplica override: donde solo_rbd_acbcd == True, fuerza objetivo a B5
objetivo_series_over = objetivo_series.copy()
objetivo_series_over.loc[solo_rbd_acbcd[solo_rbd_acbcd].index] = "B4"

# --- 3) Comparación OD a OD (usando el objetivo con override) ---
comparacion = pd.DataFrame({
    "Bundle_min": min_por_od.astype(str),
    "Bundle_objetivo": objetivo_series_over.astype(str)
})
comparacion["Match"] = (comparacion["Bundle_min"] == comparacion["Bundle_objetivo"])

# --- 4) Reportes útiles (igual que antes) ---
tot = len(comparacion)
ok = int(comparacion["Match"].sum())
nok = tot - ok
print(f"ODs evaluados: {tot} | OK: {ok} | NO OK: {nok}")

if nok:
    print("\nODs que NO cumplen (primeros 20):")
    print(comparacion[~comparacion["Match"]].head(20))

print("\nConteo por objetivo:")
print(comparacion["Bundle_objetivo"].value_counts())

min_global = str(dfb[COL_BUNDLE].min())
print(f"\nMínimo global observado: {min_global}")


ODs evaluados: 15 | OK: 15 | NO OK: 0

Conteo por objetivo:
Bundle_objetivo
B1    15
Name: count, dtype: int64

Mínimo global observado: B1


In [21]:
# --- 3.1) Listas de ODs que cumplen y no cumplen ---
comparacion = comparacion.copy()
comparacion.index.name = COL_OD
comparacion = comparacion.reset_index()  # OD pasa a columna

cumplen = comparacion.loc[comparacion["Match"]].copy()
no_cumplen = comparacion.loc[~comparacion["Match"]].copy()

print("\n=== ODs que SÍ cumplen (primeros 20) ===")
print(cumplen.head(20).to_string(index=False))

print("\n=== ODs que NO cumplen (primeros 20) ===")
print(no_cumplen.head(20).to_string(index=False))


=== ODs que SÍ cumplen (primeros 20) ===
    OD Bundle_min Bundle_objetivo  Match
LIMSEA         B1              B1   True
MSPSAL         B1              B1   True
MSPUIO         B1              B1   True
SALMSP         B1              B1   True
SALSEA         B1              B1   True
SALSLC         B1              B1   True
SALSMF         B1              B1   True
SANUIO         B1              B1   True
SEALIM         B1              B1   True
SEASAL         B1              B1   True
SEAUIO         B1              B1   True
SLCSAL         B1              B1   True
SMFSAL         B1              B1   True
UIOMSP         B1              B1   True
UIOSAN         B1              B1   True

=== ODs que NO cumplen (primeros 20) ===
Empty DataFrame
Columns: [OD, Bundle_min, Bundle_objetivo, Match]
Index: []


# Cargar archivo: Parametros offline

In [27]:
# --- Config ---
# ruta_excel ya definida al inicio en la primera celda

hoja_dist = "KMS"      # hoja donde están OD y Kms
hoja_tarifarios = "tarifarios"   # hoja con tarifarios
hoja_fuel = "FUEL OW"
hoja_SPAs = "Payouts3"
COL_OD = "ATPCO RT"  # columna con OD
COL_KMS = "Kms"

# Abrir una sola vez el archivo
xf = pd.ExcelFile(ruta_excel, engine="openpyxl")

# Leer hojas
distancias = xf.parse(hoja_dist, dtype={COL_OD: str})
tarifarios = xf.parse(hoja_tarifarios)

In [28]:
map_air2city = {
    "CDG": "PAR",
    "LHR": "LON",
    "YYZ": "YTO",
    "ORD": "CHI",
    "IAH": "HOU",
    "JFK": "NYC",
    "IAD": "WAS",
    "GIG": "RIO",
    "GRU": "SAO",
    "YUL":"YMQ",
    "CIA":"ROM"
}

In [29]:
def swap_endpoints(od: str) -> str:
    if pd.isna(od):
        return od
    s = str(od).strip().upper().rstrip(",")
    if len(s) < 3:
        return s

    # Prefijo (primeros 3)
    head = s[:3]
    if head in map_air2city:
        s = map_air2city[head] + s[3:]

    # Sufijo (últimos 3) — recomputado por si cambió antes
    tail = s[-3:]
    if tail in map_air2city:
        s = s[:-3] + map_air2city[tail]

    return s

In [30]:
tarifarios.head(5)

,TARIFARIO AGRUPADO,OD ATPCO RT
0,SAAR2,ABVBOG
1,SAAR2,ABZADZ
2,SAAR2,VLCRIO
3,SAAR2,VLCSAO
4,SAAR2,ABZBAQ


In [ ]:
# Diccionario: {"columna_original": "nuevo_nombre"}
renombres = {
    "TARIFARIO AGRUPADO": "Tariff code",
    "OD ATPCO RT": "OD",
    }

# Aplicar renombrado en df_descarga
tarifarios = tarifarios.rename(columns=renombres)

In [ ]:
# Diccionario: {"columna_original": "nuevo_nombre"}
renombres = {
    "ATPCO RT": "OD",
    }

# Aplicar renombrado en df_descarga
distancias = distancias.rename(columns=renombres)

In [ ]:
distancias["OD"] = distancias["OD"].apply(swap_endpoints)

# Validacion Tarifario

In [ ]:
# ================== Config ==================
COL_OD        = "OD"                 # Columna con OD en ambos DataFrames
COL_TAR   = "Tariff code"        # Columna de tarifario en df_filtrado_final

def norm_txt(s: pd.Series) -> pd.Series:
    """Normaliza texto: conserva NA, strip, uppercase."""
    return s.astype("string").str.strip().str.upper()

# ================== 1) Preparar 'tarifarios' (OD -> Tarifario_ref) ==================

ref = tarifarios[[COL_OD, COL_TAR]].copy()
ref[COL_OD]            = norm_txt(ref[COL_OD])
ref["Tarifario_ref"]   = norm_txt(ref[COL_TAR])
ref = ref.drop_duplicates(subset=[COL_OD], keep="first")[ [COL_OD, "Tarifario_ref"] ]


# Índice para mapeo rápido por OD
ref_map = ref.set_index(COL_OD)["Tarifario_ref"]

# ================== 2) Preparar df_filtrado_final ==================
df_check = df_filtrado_final.copy()
df_check[COL_OD]           = norm_txt(df_check[COL_OD])
df_check["Tarifario_orig"] = norm_txt(df_check[COL_TAR])

# ================== 3) Asignación por OD ==================
df_check["Tarifario_ref"] = df_check[COL_OD].map(ref_map)

# ================== 4) Completar faltantes probando ODRT =================
faltan = df_check["Tarifario_ref"].isna()
if faltan.any():
    od_rev = df_check.loc[faltan, COL_OD].str[3:] + df_check.loc[faltan, COL_OD].str[:3]
    df_check.loc[faltan, "Tarifario_ref"] = od_rev.map(ref_map).values
# ===== 4.b) Flags para vacíos y comparación (poner justo después de completar Tarifario_ref con ODRT) =====
import numpy as np

orig = df_check["Tarifario_orig"]
ref  = df_check["Tarifario_ref"]

# Hay referencia (OD u ODRT encontrados en la tabla de 'tarifarios')
mask_ref_exists = ref.notna()

# Original vacío: NaN o cadena vacía después de normalizar
mask_orig_empty = orig.isna() | (orig.str.len().fillna(0) == 0)

# Casos
mask_match      = (~mask_orig_empty) & mask_ref_exists & (orig == ref)   # OK
mask_mismatch   = (~mask_orig_empty) & mask_ref_exists & (orig != ref)   # ERROR
mask_warn_empty = mask_orig_empty & mask_ref_exists                      # ADVERTENCIA

# Columna de estado resumido
df_check["Tarifario_estado"] = np.select(
    [
        ref.isna(),        # no hay referencia (ni por OD ni por ODRT)
        mask_warn_empty,   # hay referencia pero el campo original está vacío
        mask_match         # coincide
    ],
    ["SIN_REF", "ORIG_VACIO", "OK"],
    default="MISMATCH"     # hay valor y no coincide con la referencia
)

# Para compatibilidad con el resto del código:
df_check["Tarifario_match"] = mask_match

# ================== 5) Comparación y reporte ==================
df_check["Tarifario_match"] = (df_check["Tarifario_orig"] == df_check["Tarifario_ref"])

# ODs sin tarifario en la tabla de referencia (ni directo ni inverso)
ods_sin_ref = (df_check.loc[df_check["Tarifario_ref"].isna(), COL_OD]
                        .dropna().drop_duplicates().tolist())

# Mismatches (cuando hay referencia pero no coincide)
mismatch = df_check.loc[
    df_check["Tarifario_ref"].notna() & ~df_check["Tarifario_match"],
    [COL_OD, "Tarifario_orig", "Tarifario_ref"]
].drop_duplicates()

print(f"Total filas: {len(df_check)}")
print(f"Coincidencias exactas: {df_check['Tarifario_match'].sum()} | No coincidencias: {(~df_check['Tarifario_match']).sum()}")

if ods_sin_ref:
    print(f"ODs SIN tarifario en 'tarifarios' ni por OD ni por ODRT ({len(ods_sin_ref)}):")
    print(ods_sin_ref)
else:
    print("Todos los ODs encontraron tarifario (por OD o por ODRT).")

if not mismatch.empty:
    print("\nMismatches (primeros 10):")
    print(mismatch.head(10).to_string(index=False))
else:
    print("\nNo se encontraron diferencias entre el tarifario asignado y el de referencia.")
df_resultado = df_check

Total filas: 320
Coincidencias exactas: 320 | No coincidencias: 0
Todos los ODs encontraron tarifario (por OD o por ODRT).

No se encontraron diferencias entre el tarifario asignado y el de referencia.


In [ ]:
df_filtrado_final["OD"].value_counts()

OD
ROCBOG    320
Name: count, dtype: int64

In [ ]:
df_check.head(10)

,Tariff code,Origin,Destination,Fareclass,OW/RT,Routing number,Action code,FN,Currency,Base price,Tarifa_USD,RBD,OD,Cabin,Bundle,Tarifa_inicial_USD,Tarifario_orig,Tarifario_ref,Tarifario_estado,Tarifario_match
0,WHFP,ROC,BOG,EHRB3BRS,2,414,I,NaN,USD,638.0,638.0,E,ROCBOG,Y,B3,1276.0,WHFP,WHFP,OK,True
1,WHFP,ROC,BOG,EHRB3BRS,2,412,I,NaN,USD,817.5,817.5,E,ROCBOG,Y,B3,1635.0,WHFP,WHFP,OK,True
2,WHFP,ROC,BOG,ELMB1BRS,1,414,I,NaN,USD,485.0,485.0,E,ROCBOG,Y,B1,485.0,WHFP,WHFP,OK,True
3,WHFP,ROC,BOG,ELMB1BRS,1,412,I,NaN,USD,720.0,720.0,E,ROCBOG,Y,B1,720.0,WHFP,WHFP,OK,True
4,WHFP,ROC,BOG,ELMB2BRS,1,414,I,NaN,USD,539.0,539.0,E,ROCBOG,Y,B2,539.0,WHFP,WHFP,OK,True
5,WHFP,ROC,BOG,ELMB2BRS,1,412,I,NaN,USD,775.0,775.0,E,ROCBOG,Y,B2,775.0,WHFP,WHFP,OK,True
6,WHFP,ROC,BOG,ELMB3BRS,1,414,I,NaN,USD,638.0,638.0,E,ROCBOG,Y,B3,638.0,WHFP,WHFP,OK,True
7,WHFP,ROC,BOG,ELMB3BRS,1,412,I,NaN,USD,870.0,870.0,E,ROCBOG,Y,B3,870.0,WHFP,WHFP,OK,True
8,WHFP,ROC,BOG,ELRB1BRS,2,414,I,NaN,USD,435.0,435.0,E,ROCBOG,Y,B1,870.0,WHFP,WHFP,OK,True
9,WHFP,ROC,BOG,ELRB1BRS,2,412,I,NaN,USD,617.5,617.5,E,ROCBOG,Y,B1,1235.0,WHFP,WHFP,OK,True


## Calculo Yields

Distancias

In [ ]:
COL_OD  = "OD"
COL_KMS = "KMS"   # nombre de la columna con kilómetros en 'distancias'

# --- 1) distancias: usar tal cual (solo limpiar Kms y deduplicar por OD por seguridad) ---
dist = distancias[[COL_OD, COL_KMS]].copy()
dist[COL_KMS] = pd.to_numeric(dist[COL_KMS], errors="coerce")
dist = dist.dropna(subset=[COL_OD]).drop_duplicates(subset=[COL_OD], keep="first")

# --- 2) Merge directo por OD (sin tocar distancias) ---
df_out = df_filtrado_final.copy()
df_out = df_out.merge(dist, on=COL_OD, how="left")

# --- 3) Para los que no encontraron Kms, buscar con OD invertido (solo invertimos en el filtrado) ---
faltan = df_out[COL_KMS].isna()
if faltan.any():
    # Genera OD invertido SOLO en filtrado final (AAA BBB -> BBB AAA)
    od_rev = df_out.loc[faltan, COL_OD].str[3:] + df_out.loc[faltan, COL_OD].str[:3]

    # Trae kms de distancias comparando ese OD invertido contra distancias[OD] (distancias NO se invierte)
    kms_rev = od_rev.map(dist.set_index(COL_OD)[COL_KMS])

    # Rellena los Kms faltantes con los obtenidos por el OD invertido
    df_out.loc[faltan, COL_KMS] = kms_rev.values

# --- 4) Reporte de ODs sin kms (ni directo ni inverso) ---
ods_sin_kms = (df_out.loc[df_out[COL_KMS].isna(), COL_OD]
                      .dropna()
                      .drop_duplicates()
                      .tolist())

print(f"KMS asignados: {df_out[COL_KMS].notna().sum()} / {len(df_out)} filas")
if ods_sin_kms:
    print(f"ODs sin kms ({len(ods_sin_kms)}):")
    print(ods_sin_kms)
else:
    print("Todos los ODs tienen kms.")

KMS asignados: 320 / 320 filas
Todos los ODs tienen kms.


In [ ]:
df_filtrado_final["kms"]=df_out["KMS"]

Base+fuel en dolares

In [ ]:
fuel=xf.parse(hoja_fuel)

In [ ]:
fuel["OD ATPCO OW"] = fuel["OD ATPCO OW"].apply(swap_endpoints)

In [ ]:
# ================== Config ==================

COL_OD            = "OD"
COL_FARECLASS     = "Fareclass"      # en df_filtrado_final
COL_BASE          = "Tarifa_USD"     # en df_filtrado_final (USD o moneda base de tu dataset)
COL_FUEL_OD       = "OD ATPCO OW"    # en fuel
COL_FUEL_AMT      = "FUEL OW"        # monto numérico del fuel en su moneda original
COL_FUEL_CURR     = "CURR"           # moneda del fuel (ej. 'USD','COP','EUR')
COL_TRM_USD_DICT  = dict(rates_usd)  # usa dict de TRM (USD por unidad de moneda)

# ================== 1) Preparar tabla fuel y TRM ==================
fuel_tbl = fuel[[COL_FUEL_OD, COL_FUEL_AMT, COL_FUEL_CURR]].copy()

# normaliza moneda y convierte a numérico
fuel_tbl[COL_FUEL_CURR] = fuel_tbl[COL_FUEL_CURR].astype(str).str.strip().str.upper()
fuel_tbl[COL_FUEL_AMT]  = pd.to_numeric(fuel_tbl[COL_FUEL_AMT], errors="coerce")

# mapea TRM -> USD por unidad
fuel_tbl["TRM_to_USD"]  = fuel_tbl[COL_FUEL_CURR].map(COL_TRM_USD_DICT)

# Fuel en USD
fuel_tbl["Fuel_USD"]    = fuel_tbl[COL_FUEL_AMT] * fuel_tbl["TRM_to_USD"]

# si hubiera ODs duplicados en fuel, tomamos el primero no nulo (ajusta a 'sum','mean' si corresponde)
fuel_tbl = (fuel_tbl
            .dropna(subset=[COL_FUEL_OD])
            .drop_duplicates(subset=[COL_FUEL_OD], keep="first")
            [[COL_FUEL_OD, "Fuel_USD"]]
           )

# ================== 2) Merge al df_filtrado_final ==================
df_out = df_filtrado_final.copy()
df_out = df_out.merge(fuel_tbl.rename(columns={COL_FUEL_OD: COL_OD}), on=COL_OD, how="left")

# ================== 3) Sumar fuel al base price con excepción por Fareclass ==================
# máscara de excepciones: Fareclass termina en 7, 8 o 9
mask_exento = df_out[COL_FARECLASS].astype(str).str.strip().str[-1].isin(list("789"))

# aseguremos base numérico
df_out[COL_BASE] = pd.to_numeric(df_out[COL_BASE], errors="coerce")

# Base + Fuel (si no exento; si exento, queda igual a Base)
df_out["Base_mas_Fuel"] = np.where(
    mask_exento,
    df_out[COL_BASE],  # no suma fuel
    np.where(
        df_out["Fuel_USD"].notna(),              # solo si hay Fuel
        df_out[COL_BASE] + df_out["Fuel_USD"],   # sumo
        np.nan                                   # si no hay Fuel -> sin valor
    )
)

# ================== 4) Reportes ==================
# ODs sin Fuel encontrado (Fuel_USD NaN)
ods_sin_fuel = (df_out.loc[df_out["Fuel_USD"].isna(), COL_OD]
                      .dropna().drop_duplicates().tolist())

# ODs EXENTOS por Fareclass (termina en 7/8/9)  <<-- esto es lo que quieres llamar "fuel=0" en el TXT
ods_exentos = (df_out.loc[mask_exento, COL_OD]
                      .dropna().drop_duplicates().tolist())

# (Opcional) ODs con Fuel_USD = 0 real (por si lo quieres reportar aparte)
ods_fuel_cero_real = (df_out.loc[df_out["Fuel_USD"].fillna(np.nan) == 0, COL_OD]
                             .dropna().drop_duplicates().tolist())

print(f"Filas con Fuel mapeado a USD: {df_out['Fuel_USD'].notna().sum()} / {len(df_out)}")
if ods_sin_fuel:
    print(f"ODs SIN fuel (no se encontró o TRM faltante) [{len(ods_sin_fuel)}]:")
    print(ods_sin_fuel)
else:
    print("Todos los ODs tienen algún valor de Fuel_USD (no NaN).")

print(f"Filas EXENTAS por Fareclass 7/8/9: {int(mask_exento.sum())}")
if ods_exentos:
    print(f"ODs EXENTOS (tratados como 'fuel=0' en el TXT) [{len(ods_exentos)}]:")
    print(ods_exentos[:30])

# Chequeo de consistencia: en exentos, Base_mas_Fuel debe ser igual a Base
ok_exentos = (df_out.loc[mask_exento, "Base_mas_Fuel"] == df_out.loc[mask_exento, COL_BASE]).all()
print(f"Chequeo exentos (Base_mas_Fuel == Base): {'OK' if ok_exentos else 'REVISAR'}")

Filas con Fuel mapeado a USD: 320 / 320
Todos los ODs tienen algún valor de Fuel_USD (no NaN).
Filas EXENTAS por Fareclass 7/8/9: 16
ODs EXENTOS (tratados como 'fuel=0' en el TXT) [1]:
['ROCBOG']
Chequeo exentos (Base_mas_Fuel == Base): OK


In [ ]:
df_out["Fuel_USD"].head(10)

0    77.0
1    77.0
2    77.0
3    77.0
4    77.0
5    77.0
6    77.0
7    77.0
8    77.0
9    77.0
Name: Fuel_USD, dtype: float64

In [ ]:
df_out["Fuel_USD"].value_counts()

Fuel_USD
77.0    320
Name: count, dtype: int64

In [ ]:
df_out.head(10)

,Tariff code,Origin,Destination,Fareclass,OW/RT,Routing number,Action code,FN,Currency,Base price,Tarifa_USD,RBD,OD,Cabin,Bundle,Tarifa_inicial_USD,kms,Fuel_USD,Base_mas_Fuel
0,WHFP,ROC,BOG,EHRB3BRS,2,414,I,NaN,USD,638.0,638.0,E,ROCBOG,Y,B3,1276.0,4285.905874,77.0,715.0
1,WHFP,ROC,BOG,EHRB3BRS,2,412,I,NaN,USD,817.5,817.5,E,ROCBOG,Y,B3,1635.0,4285.905874,77.0,894.5
2,WHFP,ROC,BOG,ELMB1BRS,1,414,I,NaN,USD,485.0,485.0,E,ROCBOG,Y,B1,485.0,4285.905874,77.0,562.0
3,WHFP,ROC,BOG,ELMB1BRS,1,412,I,NaN,USD,720.0,720.0,E,ROCBOG,Y,B1,720.0,4285.905874,77.0,797.0
4,WHFP,ROC,BOG,ELMB2BRS,1,414,I,NaN,USD,539.0,539.0,E,ROCBOG,Y,B2,539.0,4285.905874,77.0,616.0
5,WHFP,ROC,BOG,ELMB2BRS,1,412,I,NaN,USD,775.0,775.0,E,ROCBOG,Y,B2,775.0,4285.905874,77.0,852.0
6,WHFP,ROC,BOG,ELMB3BRS,1,414,I,NaN,USD,638.0,638.0,E,ROCBOG,Y,B3,638.0,4285.905874,77.0,715.0
7,WHFP,ROC,BOG,ELMB3BRS,1,412,I,NaN,USD,870.0,870.0,E,ROCBOG,Y,B3,870.0,4285.905874,77.0,947.0
8,WHFP,ROC,BOG,ELRB1BRS,2,414,I,NaN,USD,435.0,435.0,E,ROCBOG,Y,B1,870.0,4285.905874,77.0,512.0
9,WHFP,ROC,BOG,ELRB1BRS,2,412,I,NaN,USD,617.5,617.5,E,ROCBOG,Y,B1,1235.0,4285.905874,77.0,694.5


In [ ]:
df_filtrado_final["Base_mas_Fuel"] = df_out["Base_mas_Fuel"]

In [ ]:
df_filtrado_final.head(10)

,Tariff code,Origin,Destination,Fareclass,OW/RT,Routing number,Action code,FN,Currency,Base price,Tarifa_USD,RBD,OD,Cabin,Bundle,Tarifa_inicial_USD,kms,Base_mas_Fuel
0,WHFP,ROC,BOG,EHRB3BRS,2,414,I,NaN,USD,638.0,638.0,E,ROCBOG,Y,B3,1276.0,4285.905874,715.0
1,WHFP,ROC,BOG,EHRB3BRS,2,412,I,NaN,USD,817.5,817.5,E,ROCBOG,Y,B3,1635.0,4285.905874,894.5
2,WHFP,ROC,BOG,ELMB1BRS,1,414,I,NaN,USD,485.0,485.0,E,ROCBOG,Y,B1,485.0,4285.905874,562.0
3,WHFP,ROC,BOG,ELMB1BRS,1,412,I,NaN,USD,720.0,720.0,E,ROCBOG,Y,B1,720.0,4285.905874,797.0
4,WHFP,ROC,BOG,ELMB2BRS,1,414,I,NaN,USD,539.0,539.0,E,ROCBOG,Y,B2,539.0,4285.905874,616.0
5,WHFP,ROC,BOG,ELMB2BRS,1,412,I,NaN,USD,775.0,775.0,E,ROCBOG,Y,B2,775.0,4285.905874,852.0
6,WHFP,ROC,BOG,ELMB3BRS,1,414,I,NaN,USD,638.0,638.0,E,ROCBOG,Y,B3,638.0,4285.905874,715.0
7,WHFP,ROC,BOG,ELMB3BRS,1,412,I,NaN,USD,870.0,870.0,E,ROCBOG,Y,B3,870.0,4285.905874,947.0
8,WHFP,ROC,BOG,ELRB1BRS,2,414,I,NaN,USD,435.0,435.0,E,ROCBOG,Y,B1,870.0,4285.905874,512.0
9,WHFP,ROC,BOG,ELRB1BRS,2,412,I,NaN,USD,617.5,617.5,E,ROCBOG,Y,B1,1235.0,4285.905874,694.5


## Validación minimos

### Calculo yields

In [ ]:
df_filtrado_final["Yield"]= (df_filtrado_final["Base_mas_Fuel"]*100)/(df_filtrado_final["kms"])

In [ ]:

# ============== Config columnas ==============
COL_OD        = "OD"
COL_CABINA    = "Cabin"           
COL_TARIFA    = "Tarifa_USD"       
COL_BASEFUEL  = "Base_mas_Fuel"    
COL_DISC      = "Discontinue date"
COL_NUM       = "Tariff code"
COL_FN        = "FN"
COL_RBD       = "RBD"
COL_CURR      = "Currency"
COL_FAREBASIS = "Fareclass"

# ============== Limpieza básica ==============
# Asegura numérico en Tarifa y en Base+Fuel (por si quieres ver el valor en la misma fila)
df_filtrado_final[COL_TARIFA]   = pd.to_numeric(df_filtrado_final[COL_TARIFA],   errors="coerce")
df_filtrado_final[COL_BASEFUEL] = pd.to_numeric(df_filtrado_final[COL_BASEFUEL], errors="coerce")

df_open = df_filtrado_final.copy()

# Quita filas sin Tarifa_USD (NaN) para poder calcular el mínimo
df_open = df_open.dropna(subset=[COL_TARIFA]).copy()

# ============== Cálculo del mínimo por (OD, Cabin) usando Tarifa_USD ==============
idx_min = df_open.groupby([COL_OD, COL_CABINA], dropna=False)[COL_TARIFA].idxmin()
min_rows = df_open.loc[idx_min].copy()

# Agrega columna con el Base+Fuel correspondiente a la misma fila mínima por tarifa
min_rows["BaseFuel_en_MinTarifa"] = min_rows[COL_BASEFUEL]

# Renombrar columnas de interés (el resto se conserva tal cual)
min_rows = min_rows.rename(columns={
    COL_TARIFA:   "Min_Tarifa_USD", 
    COL_NUM:      "Tariff_code_min",
    COL_RBD:      "RBD_min",
    COL_CURR:     "Currency_min",
    COL_FAREBASIS:"Fareclass_min"
})

# ============== Exportar ==============
resumen_6cols = min_rows[[COL_OD, COL_CABINA, "Min_Tarifa_USD", "BaseFuel_en_MinTarifa",
                          "Tariff_code_min", "RBD_min", "Fareclass_min", "Currency_min"]].copy()
resumen_6cols = resumen_6cols.sort_values([COL_OD, COL_CABINA])

# (B) Resumen completo con TODAS las columnas de la fila mínima (opcional)
resumen_completo = min_rows.sort_values([COL_OD, COL_CABINA])

# --- Escribir ---
if not resumen_6cols.empty:
    write_sheet(resumen_6cols, excel_path, sheet_name="Resumen_OD_Cabin_Min", replace=True)
else:
    df_empty = pd.DataFrame({"msg": ["SIN RESUMEN (todas las filas tienen Tarifa_USD = NaN)"]})
    write_sheet(df_empty, excel_path, sheet_name="Resumen_OD_Cabin_Min", replace=True)

print("Excel creado en:", excel_path)

Excel creado en: C:\Users\amvillamizar\Downloads\Avianca\Offline\Documentos\Publicacion tarifas\Pricing_checks\Parametros_Offline\Resumen\140126_PRUEBA_ABS.xlsx


In [ ]:
df_filtrado_final.head(5)

,Tariff code,Origin,Destination,Fareclass,OW/RT,Routing number,Action code,FN,Currency,Base price,Tarifa_USD,RBD,OD,Cabin,Bundle,Tarifa_inicial_USD,kms,Base_mas_Fuel,Yield
0,WHFP,ROC,BOG,EHRB3BRS,2,414,I,NaN,USD,638.0,638.0,E,ROCBOG,Y,B3,1276.0,4285.905874,715.0,16.682588
1,WHFP,ROC,BOG,EHRB3BRS,2,412,I,NaN,USD,817.5,817.5,E,ROCBOG,Y,B3,1635.0,4285.905874,894.5,20.870734
2,WHFP,ROC,BOG,ELMB1BRS,1,414,I,NaN,USD,485.0,485.0,E,ROCBOG,Y,B1,485.0,4285.905874,562.0,13.112747
3,WHFP,ROC,BOG,ELMB1BRS,1,412,I,NaN,USD,720.0,720.0,E,ROCBOG,Y,B1,720.0,4285.905874,797.0,18.595835
4,WHFP,ROC,BOG,ELMB2BRS,1,414,I,NaN,USD,539.0,539.0,E,ROCBOG,Y,B2,539.0,4285.905874,616.0,14.372691


In [ ]:
# Supongamos que tus columnas se llaman:
COL_CABIN  = "Cabin" 
COL_YIELD  = "Yield"   

# --- Validación ---
df_check = df_filtrado_final.copy()
df_check[COL_YIELD] = pd.to_numeric(df_check[COL_YIELD], errors="coerce")

# Condiciones por cabina
cond_c1 = (df_check[COL_CABIN] == "Y") & (df_check[COL_YIELD] > 2)
cond_c2 = (df_check[COL_CABIN] == "C") & (df_check[COL_YIELD] > 6)

df_check["Cumple"] = cond_c1 | cond_c2

# --- Filas que no cumplen ---
df_no_cumplen = df_check.loc[~df_check["Cumple"]].drop(columns=["Cumple"])

if df_no_cumplen.empty:
    print("✅ Todas las tarifas cumplen con los mínimos de yield.")
else:
    print(f"⚠️ Hay {len(df_no_cumplen)} tarifas que NO cumplen con los mínimos de yield.")
    display(df_no_cumplen)

✅ Todas las tarifas cumplen con los mínimos de yield.


In [ ]:
# Helper de listado corto para el markdown
def top_list(values, n=10):
    vals = list(values)
    if not vals:
        return "- (ninguno)\n"
    head = vals[:n]
    s = "".join([f"- {x}\n" for x in head])
    if len(vals) > n:
        s += f"- ... y {len(vals)-n} más\n"
    return s

# ========== 0) Validación OW/RT por 3er carácter de Fareclass ==========
owrt_lines = ["0. Validación OW/RT por Fareclass (3er carácter)\n"]

# Asegurar formato de Fareclass
_dfow = df_filtrado_final  # usa el DF principal
_dfow["Fareclass"] = _dfow["Fareclass"].astype(str).str.strip().str.upper()

# Tercer carácter
third_char = _dfow["Fareclass"].str[2]

# Máscaras
mask_A = third_char.eq("M")
mask_R = third_char.eq("R")

# Guardar valores previos (opcional, para auditar cambios)
owrt_prev = _dfow["OW/RT"] if "OW/RT" in _dfow.columns else pd.Series([pd.NA]*len(_dfow), index=_dfow.index)

# Aplicar regla
_dfow.loc[mask_A, "OW/RT"] = 1
_dfow.loc[mask_R, "OW/RT"] = 2

# Validación de consistencia
fails_A = _dfow.loc[mask_A & (_dfow["OW/RT"] != 1), ["OD","Fareclass","OW/RT"]]
fails_M = _dfow.loc[mask_R & (_dfow["OW/RT"] != 2), ["OD","Fareclass","OW/RT"]]

if fails_A.empty and fails_M.empty:
    owrt_lines.append("✅ El Fareclass es coherente con la asignación OW/RT en todas las filas.\n\n")
else:
    total_fails = len(fails_A) + len(fails_M)
    owrt_lines.append(f"❌ {total_fails} filas NO cumplen la regla OW/RT según el 3er carácter.\n\n")
# ---------- 1) Validación corners (usa 'violaciones' ya calculado) ----------
corners_lines = ["1. Validación corners (reglas Fareclass)\n"]
corners_n = None
if "violaciones" in globals():
    corners_n = len(violaciones)
    if corners_n == 0:
        corners_lines.append("✅ **Todas las filas cumplen** las reglas de corners.\n\n")
    else:
        corners_lines.append(f"⚠️ **{corners_n} filas** incumplen las reglas de corners.\n\n")
        # opcional: ejemplos
        corners_lines.append("Ejemplos (primeros 15 Fareclass):\n")
        corners_lines.append(top_list(violaciones["Fareclass"].dropna().astype(str).unique().tolist(), 15)+"\n\n")
else:
    corners_lines.append("ℹ️ Este chequeo no se ejecutó en esta corrida (no existe `violaciones`).\n\n")

# ---------- 2) Bundle mínimo (usa 'comparacion') ----------
bundle_lines = ["2. Bundle mínimo vs objetivo por OD\n"]
bundle_nok = None
if "comparacion" in globals():
    if {"Bundle_min","Bundle_objetivo","Match"}.issubset(comparacion.columns):
        tot_b = len(comparacion)
        bundle_nok = int((~comparacion["Match"]).sum())
        if bundle_nok == 0:
            bundle_lines.append(f"✅ **Todos los {tot_b} ODs** cumplen el bundle objetivo.\n\n")
        else:
            bundle_lines.append(f"⚠️ **{bundle_nok} de {tot_b} ODs** NO cumplen el bundle objetivo (ver hoja `Bundles`).\n\n")
    else:
        bundle_lines.append("⚠️ `comparacion` no tiene las columnas esperadas (Bundle_min/Bundle_objetivo/Match).\n\n")
else:
    bundle_lines.append("ℹ️ Este chequeo no se ejecutó en esta corrida (no existe `comparacion`).\n\n")

# ---------- 3) Validación de tarifario (OD → referencia) ----------
tarif_lines = ["3. Validación de tarifario (OD → referencia)\n"]
tarif_df = None
if "df_resultado" in globals():
    tarif_df = df_resultado
elif "df_check" in globals():
    tarif_df = df_check

if tarif_df is not None and "Tarifario_estado" in tarif_df.columns:
    tot_t   = len(tarif_df)
    n_ok    = int((tarif_df["Tarifario_estado"] == "OK").sum())
    n_warn  = int((tarif_df["Tarifario_estado"] == "ORIG_VACIO").sum())
    n_err   = int((tarif_df["Tarifario_estado"] == "MISMATCH").sum())

    # Mensaje principal
    if n_err == 0 and n_warn == 0:
        tarif_lines.append(f"✅ **Todos los {tot_t} registros** coinciden con el tarifario de referencia.\n\n")
    else:
        if n_err > 0:
            df_err = tarif_df.loc[tarif_df["Tarifario_estado"] == "MISMATCH", [COL_OD, "Tarifario_orig", "Tarifario_ref"]].drop_duplicates()
            tarif_lines.append(f"❌ **{n_err} de {tot_t}** registros NO coinciden con el tarifario de referencia.\n")
            tarif_lines.append("ODs con discrepancia (muestra):\n")
            tarif_lines.append(top_list(df_err[COL_OD].drop_duplicates().tolist(), n=15) + "\n")

        if n_warn > 0:
            df_warn = tarif_df.loc[tarif_df["Tarifario_estado"] == "ORIG_VACIO", [COL_OD, "Tarifario_ref"]].drop_duplicates()
            tarif_lines.append(f"⚠️ **{n_warn} registros** tienen el campo de tarifario vacío.\n")
            tarif_lines.append("ODs con campo vacío (muestra):\n")
            tarif_lines.append(top_list(df_warn[COL_OD].drop_duplicates().tolist(), n=15) + "\n")

    # (Opcional) resumen de conteos
    tarif_lines.append(f"- OK: {n_ok} | Advertencias (vacío): {n_warn} | Errores (mismatch): {n_err}\n\n")

else:
    tarif_lines.append("ℹ️ Este chequeo no se ejecutó en esta corrida (no existe `df_resultado/df_check` con `Tarifario_estado`).\n\n")

# ---------- 4) Validación de Yield (Y>2, C>6). Usa df_no_cumplen si ya existe; si no, lo calcula rápido ----------
yield_lines = ["6. Validación de yield por cabina\n- Regla cabina Y: Yield > 2\n- Regla cabina C: Yield > 6\n"]
_dfy = df_filtrado_final.copy()
_dfy["Yield"] = pd.to_numeric(_dfy["Yield"], errors="coerce")
cab = _dfy["Cabin"].astype(str).str.upper()

is_Y = cab.eq("Y")
is_C = cab.eq("C")
consideradas = (is_Y | is_C)

mask_missing = consideradas & _dfy["Yield"].isna()
mask_bajo    = (is_Y & _dfy["Yield"].notna() & (_dfy["Yield"] <= 2)) | \
               (is_C & _dfy["Yield"].notna() & (_dfy["Yield"] <= 6))
mask_ok      = (is_Y & (_dfy["Yield"] > 2)) | (is_C & (_dfy["Yield"] > 6))

tot_consideradas = int(consideradas.sum())
cnt_missing = int(mask_missing.sum())
cnt_bajo    = int(mask_bajo.sum())
cnt_ok      = int(mask_ok.sum())

yield_lines.append(f"- ✅ Cumplen threshold: {cnt_ok}\n")
# Agregar advertencias solo si hay
if cnt_bajo > 0:
    yield_lines.append(f"- ⚠️ No cumplen por **yield bajo**: {cnt_bajo}\n")
if cnt_missing > 0:
    yield_lines.append(f"- ⚠️ No cumplen por **dato faltante (Yield NaN)**: {cnt_missing}\n")

# Si no hubo advertencias, agregar línea positiva
if cnt_bajo == 0 and cnt_missing == 0:
    yield_lines.append("- ✅ Todos los OD cumplen el threshold\n")

yield_lines.append("\n")

# (Opcional) Listar ejemplos de ODs para cada caso en el TXT
ods_bajo = _dfy.loc[mask_bajo, "OD"].dropna().drop_duplicates().tolist()
ods_missing = _dfy.loc[mask_missing, "OD"].dropna().drop_duplicates().tolist()

yield_lines.append("ODs con yield por debajo del umbral (muestra):\n")
yield_lines.append(top_list(ods_bajo, n=20) + "\n")

yield_lines.append("ODs sin dato de Yield (muestra):\n")
yield_lines.append(top_list(ods_missing, n=20) + "\n")

# 5) KMS — integra tu validación para listar ODs sin kms
kms_lines = ["4. KMS (distancias)\n"]
if ods_sin_kms:
    kms_lines.append(f"⚠️- ODs sin KMS: {len(ods_sin_kms)}\n")
    kms_lines.append(top_list(ods_sin_kms, n=30)+"\n\n")
else:
    kms_lines.append("- ✅ Todos los ODs tienen KMS.\n\n")
def top_list(vals, n=30):
    vals = list(vals or [])
    out = ", ".join(vals[:n])
    if len(vals) > n:
        out += " ..."
    return out

fuel_lines = ["5. Fuel (conversión a USD y suma)\n"]

# a) ODs sin Fuel_USD (NaN)
if ods_sin_fuel:
    fuel_lines.append(f"⚠️ ODs SIN fuel (Fuel_USD NaN): {len(ods_sin_fuel)}\n")
    fuel_lines.append(top_list(ods_sin_fuel, n=30) + "\n\n")
else:
    fuel_lines.append("✅ Todos los ODs tienen Fuel_USD (no NaN).\n\n")

# b) ODs EXENTOS por fareclass (esto será tu 'fuel=0' en el TXT)
if ods_exentos:
    fuel_lines.append(f"ℹ️ ODs con fuel=0 (exentos por Fareclass 7/8/9): {len(ods_exentos)}\n")
    fuel_lines.append(top_list(ods_exentos, n=30) + "\n\n")
else:
    fuel_lines.append("ℹ️ No hay ODs exentos por Fareclass 7/8/9.\n\n")

# (Opcional) c) ODs con Fuel_USD == 0 real
if ods_fuel_cero_real:
    fuel_lines.append(f"ℹ️ ODs con Fuel_USD=0 real: {len(ods_fuel_cero_real)}\n")
    fuel_lines.append(top_list(ods_fuel_cero_real, n=30) + "\n\n")


In [ ]:
third_char.value_counts()

Fareclass
R    160
M    160
Name: count, dtype: int64

In [ ]:
def remove_sheet_if_exists(path, sheet_name):
    """Elimina una hoja existente del Excel si existe (no falla si no hay archivo/hoja)."""
    path = Path(path)
    if not path.exists():
        return
    wb = load_workbook(path)
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        wb.remove(ws)
        # openpyxl exige al menos 1 hoja
        if len(wb.sheetnames) == 0:
            wb.create_sheet("Sheet1")
        wb.save(path)

# ===== Exportar Bundles SOLO si hay errores =====
if "comparacion" in globals():
    try:
        comp_out = comparacion.copy()
        comp_out.reset_index(inplace=True)
        comp_out.rename(columns={"index": "OD"}, inplace=True)

        hay_errores_bundles = (comp_out["Match"] == False).any()

        if hay_errores_bundles:
            comp_to_export = comp_out.loc[comp_out["Match"] == False].copy()
            write_sheet(comp_to_export, excel_path, sheet_name="Bundles", replace=True)
        else:
            remove_sheet_if_exists(excel_path, "Bundles")
            print("✅ Bundles: sin errores. No se genera hoja 'Bundles'.")
    except Exception as e:
        print("⚠️ No se pudo procesar 'Bundles':", e)

# ===== Exportar Tarifarios SOLO si hay errores =====
tarif_df = None
if "df_resultado" in globals():
    tarif_df = df_resultado
elif "df_check" in globals():
    tarif_df = df_check

if tarif_df is not None and "Tarifario_match" in tarif_df.columns:
    try:
        hay_errores_tarif = (tarif_df["Tarifario_match"] == False).any()

        if hay_errores_tarif:
            tarif_to_export = tarif_df.loc[tarif_df["Tarifario_match"] == False].copy()

            cols_prior = ["OD", "Tarifario_orig", "Tarifario_ref", "Tarifario_match"]
            cols_prior = [c for c in cols_prior if c in tarif_to_export.columns]
            other_cols = [c for c in tarif_to_export.columns if c not in cols_prior]
            tarif_to_export = tarif_to_export[cols_prior + other_cols]

            write_sheet(tarif_to_export, excel_path, sheet_name="Tarifarios", replace=True)
        else:
            remove_sheet_if_exists(excel_path, "Tarifarios")
            print("✅ Tarifarios: sin errores. No se genera hoja 'Tarifarios'.")
    except Exception as e:
        print("⚠️ No se pudo procesar 'Tarifarios':", e)
else:
    print("ℹ️ No existe 'df_resultado/df_check' con la columna 'Tarifario_match'.")

# ===== Exportar Yield (solo no cumplen) =====
if "df_no_cumplen" in globals() and df_no_cumplen is not None and len(df_no_cumplen) > 0:
    try:
        write_sheet(df_no_cumplen, excel_path, sheet_name="Yield_no_cumplen", replace=True)
    except Exception as e:
        print("⚠️ No se pudo escribir 'Yield_no_cumplen':", e)
else:
    remove_sheet_if_exists(excel_path, "Yield_no_cumplen")
    print("✅ Yield: sin errores. No se genera hoja 'Yield_no_cumplen'.")

✅ Bundles: sin errores. No se genera hoja 'Bundles'.
✅ Tarifarios: sin errores. No se genera hoja 'Tarifarios'.
✅ Yield: sin errores. No se genera hoja 'Yield_no_cumplen'.


## Calculo yields after payout

In [ ]:
RTGS_NEW  = "Consolidado RTG offline" 
xf = pd.ExcelFile(ruta_excel, engine="openpyxl")
RTG_new = xf.parse(RTGS_NEW)

In [ ]:
def _norm(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.upper()

def _pick_col(df: pd.DataFrame, candidates, required=True):
    """
    Devuelve el primer nombre de columna existente en df que coincida en candidates.
    Si required=True y no encuentra, lanza KeyError con lista de faltantes.
    """
    for c in candidates:
        if c in df.columns:
            return c
    if required:
        raise KeyError(f"Falta alguna de las columnas {candidates}")
    return None

def build_rtg_map_from_consolidado(RTG_new: pd.DataFrame) -> pd.DataFrame:
    """
    Devuelve DF con columnas: ['LegMetro','Partner','RTG'] (una fila por RTG).
    Lee de la hoja 'Consolidado RTG' con nombres variables.
    """
    t2 = RTG_new.copy()

    # Columnas base (detecta español/inglés y sector/metro)
    col_offline = _pick_col(t2, ["Endpoint Offline", "Endpoint offline"])
    col_online  = _pick_col(t2, ["Endpoint Online", "Endpoint online"])
    col_rtg     = _pick_col(t2, ["RTG", "Routing", "Routing number"])
    col_partner = _pick_col(t2, ["Partner", "Carrier", "Marketing Carrier"], required=False)

    # Columnas Unnamed que completan "Endpoint Offline" (como en tu consolidado)
    offline_extra = [c for c in t2.columns if str(c).startswith("Unnamed")]
    t2[col_offline] = (
        t2[[col_offline] + offline_extra]
          .astype(str)
          .apply(lambda row: "/".join([x for x in row if x.lower() != "nan" and x != ""]), axis=1)
    )
    if offline_extra:
        t2.drop(columns=offline_extra, inplace=True)

    # Normaliza
    # Normaliza endpoints
    for c in [col_offline, col_online]:
        t2[c] = _norm(t2[c])

    # RTG como numérico
    t2[col_rtg] = pd.to_numeric(t2[col_rtg], errors="coerce")
    t2[col_rtg] = t2[col_rtg].round().astype("Int64")
        
    if col_partner:
        t2[col_partner] = _norm(t2[col_partner])
    else:
        t2[col_partner] = ""

    # Explode de la lista offline, separada por "/"
    t2_exp = t2.assign(Offline=t2[col_offline].str.split("/")).explode("Offline")
    t2_exp["Offline"] = _norm(t2_exp["Offline"])
    t2_exp = t2_exp[(t2_exp["Offline"].notna()) & (t2_exp["Offline"] != "")]

    # Arma LegMetro en AB y BA
    AB = t2_exp.copy(); AB["LegMetro"] = AB[col_online]  + AB["Offline"]
    BA = t2_exp.copy(); BA["LegMetro"] = BA["Offline"]   + BA[col_online]
    pairs = pd.concat([AB, BA], ignore_index=True)

    pairs_key = (
        pairs[["LegMetro", col_partner, col_rtg]]
        .rename(columns={col_partner: "Partner", col_rtg: "RTG"})
        .dropna(subset=["LegMetro","RTG"])
        .drop_duplicates()
    )
    return pairs_key # columnas: LegMetro, Partner, RTG

In [ ]:
def normalize_spa_columns(spa_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Devuelve SPA normalizado con columnas:
      ['spa_leg','RBD','SPA_avg','SPA_max','SPA_min','Partner'(opc)]
    Soporta nombres español/inglés y 'Sector'/'Metro'.
    """
    df = spa_raw.copy()

    # Leg: prioriza Metro; si no existe usa Sector
    col_leg = _pick_col(df, ["Leg Offline (Metro)", "Leg Offline (Sector)", "Leg Offline"])
    # RBD
    col_rbd = _pick_col(df, ["AV RBD", "RBD", "Fare RBD"])
    # Partner (opcional)
    col_partner = _pick_col(df, ["Partner", "Carrier", "Marketing Carrier"], required=False)

    # Average / Max / Min (varias variantes)
    avg_candidates = [
        "Promedio de Average SPA", "Average SPA", "Average", "Promedio SPA"
    ]
    max_candidates = [
        "Máx. de SPA PayoutAmount USD MAX", "SPA PayoutAmount USD MAX", "Max SPA", "MAX SPA"
    ]
    min_candidates = [
        "Mín. de SPA PayoutAmount USD MIN", "SPA PayoutAmount USD MIN", "Min SPA", "MIN SPA"
    ]

    col_avg = _pick_col(df, avg_candidates)
    col_max = _pick_col(df, max_candidates)
    col_min = _pick_col(df, min_candidates, required=False)

    out = pd.DataFrame({
        "spa_leg": _norm(df[col_leg]),
        "RBD":     _norm(df[col_rbd]),
        "SPA_avg": pd.to_numeric(df[col_avg], errors="coerce"),
        "SPA_max": pd.to_numeric(df[col_max], errors="coerce"),
    })
    if col_min:
        out["SPA_min"] = pd.to_numeric(df[col_min], errors="coerce")
    if col_partner:
        out["Partner"] = _norm(df[col_partner])

    return out

In [ ]:
def attach_rtgs_to_spa(spa_norm: pd.DataFrame, rtg_map: pd.DataFrame) -> pd.DataFrame:
    """
    Adjunta RTG a cada fila de SPA.
    - Si SPA trae Partner: intenta (leg, partner) y, para las filas sin match, cae a (leg).
    - Si SPA no trae Partner: cruza por (leg).
    Devuelve un DF 'spa_exp' con posible 1→N (varias filas por RTG aplicable).
    """
    spa = spa_norm.copy()
    spa["spa_leg"] = spa["spa_leg"].astype(str).str.upper().str.replace("-", "", regex=False).str.strip()
    spa["_leg_key"] = spa["spa_leg"]

    mp = rtg_map.copy()
    mp["LegMetro"] = mp["LegMetro"].astype(str).str.upper().str.replace("-", "", regex=False).str.strip()
    mp["_leg_key"] = mp["LegMetro"]
    mp["_carrier_key"] = mp["Partner"].astype(str).str.upper().str.strip() if "Partner" in mp.columns else ""

    has_partner = "Partner" in spa.columns
    if has_partner:
        spa["_carrier_key"] = spa["Partner"].astype(str).str.upper().str.strip()

        # 1) matches por (leg, partner)
        spa["_rowid"] = np.arange(len(spa))
        m_partner = spa.merge(
            mp[["_leg_key","_carrier_key","RTG"]].drop_duplicates(),
            on=["_leg_key","_carrier_key"],
            how="inner",   # inner: solo los que matchean
            validate="m:m"
        )

        # 2) fallback por (leg) SOLO para las filas sin match por partner
        rows_con_match = set(m_partner["_rowid"].unique())
        spa_sin_match = spa.loc[~spa["_rowid"].isin(rows_con_match)].copy()

        m_leg = spa_sin_match.merge(
            mp[["_leg_key","RTG"]].drop_duplicates(),
            on="_leg_key",
            how="inner",
            validate="m:m"
        )

        spa_exp = pd.concat([m_partner, m_leg], ignore_index=True)

        # limpia helpers
        spa_exp.drop(columns=[c for c in ["_rowid","_leg_key","_carrier_key"] if c in spa_exp.columns], inplace=True)

    else:
        # Sin partner: directo por leg
        spa_exp = spa.merge(
            mp[["_leg_key","RTG"]].drop_duplicates(),
            on="_leg_key",
            how="inner",
            validate="m:m"
        )
        spa_exp.drop(columns=[c for c in ["_leg_key"] if c in spa_exp.columns], inplace=True)

    return spa_exp



In [ ]:
# ========= Función: preparar tablas SPA por rol con filtro opcional de endpoint =========
def preparar_spa_por_rol(spa_raw: pd.DataFrame, endpoint_exacto: str | None = None):
    """
    Devuelve (spa_dest_pick, spa_orig_pick) listos para hacer merge por OFFLINE_EP + RBD.
    Si endpoint_exacto es None -> usa todo el SPA.
    Si endpoint_exacto = "BOG" (p.ej.) -> filtra SPA para legs que tengan BOG como start o end.
    """
    # ---- columnas esperadas en spa_raw ----
    # "Leg Offline (Metro)" (spa_leg), "AV RBD", "Promedio de Average SPA" (SPA_avg),
    # "Máx. de SPA PayoutAmount USD MAX" (SPA_max), opcional "Mín. de SPA PayoutAmount USD MIN" (SPA_min)
    spa_cols_base = {
        "Leg Offline (Metro)": "spa_leg",
        "AV RBD": "RBD",
        "Promedio de Average SPA": "SPA_avg",
        "Máx. de SPA PayoutAmount USD MAX": "SPA_max",
        "Mín. de SPA PayoutAmount USD MIN": "SPA_min",
    }

    # detectar si hay columna de min con nombres raros
    min_candidates = [c for c in spa_raw.columns if "min" in c.lower() and "spa" in c.lower()]
    spa_min_col = min_candidates[0] if min_candidates else None

    use_cols = {k: v for k, v in spa_cols_base.items() if k in spa_raw.columns}
    if spa_min_col and "Mín. de SPA PayoutAmount USD MIN" not in use_cols:
        use_cols[spa_min_col] = "SPA_min"

    # sanity check mínimo
    for req in ["Leg Offline (Metro)", "AV RBD", "Promedio de Average SPA", "Máx. de SPA PayoutAmount USD MAX"]:
        if req not in spa_raw.columns:
            raise KeyError(f"Falta columna en SPA: {req}")

    # ---- normaliza y arma campos auxiliares ----
    spa_df = spa_raw[list(use_cols.keys())].copy().rename(columns=use_cols)
    spa_df["spa_leg"] = spa_df["spa_leg"].astype(str).str.upper().str.strip()
    spa_df["RBD"]     = spa_df["RBD"].astype(str).str.upper().str.strip()

    # separa inicio/fin del leg (IATA)
    spa_df["spa_start"] = spa_df["spa_leg"].str[:3]
    spa_df["spa_end"]   = spa_df["spa_leg"].str[-3:]

    # ---- FILTRO OPCIONAL por endpoint exacto ----
    if endpoint_exacto:
        EP = str(endpoint_exacto).strip().upper()[:3]
        spa_df = spa_df.loc[(spa_df["spa_start"] == EP) | (spa_df["spa_end"] == EP)].copy()

    # ---- agregadores (se calculan TODOS sobre el spa_df ya filtrado) ----
    agg_dict = {
        "SPA_avg": ("SPA_avg", lambda x: round(pd.to_numeric(x, errors="coerce").mean(), 2)),
        "SPA_max": ("SPA_max", lambda x: round(pd.to_numeric(x, errors="coerce").max(), 2)),
        "SPA_legs_usados": ("spa_leg", lambda s: ",".join(sorted(s.astype(str).str.upper().unique()))),
    }
    if "SPA_min" in spa_df.columns:
        agg_dict["SPA_min"] = ("SPA_min", lambda x: round(pd.to_numeric(x, errors="coerce").min(), 2))

    # ---- tablas por rol ----
    spa_dest_pick = (
        spa_df.groupby(["spa_end", "RBD"], dropna=False)
              .agg(**agg_dict)
              .reset_index()
              .rename(columns={"spa_end": "OFFLINE_EP"})
    )

    spa_orig_pick = (
        spa_df.groupby(["spa_start", "RBD"], dropna=False)
              .agg(**agg_dict)
              .reset_index()
              .rename(columns={"spa_start": "OFFLINE_EP"})
    )

    return spa_dest_pick, spa_orig_pick


In [ ]:
def preparar_spa_por_rol_con_rtg(spa_with_rtg: pd.DataFrame):
    """
    Devuelve (spa_dest_pick, spa_orig_pick) para merge por (OFFLINE_EP, RBD, RTG).
    Requiere columnas: ['spa_leg','RBD','SPA_avg','SPA_max','(SPA_min)','RTG'].
    """
    spa_df = spa_with_rtg.copy()
    for req in ["spa_leg","RBD","SPA_avg","SPA_max","RTG"]:
        if req not in spa_df.columns:
            raise KeyError(f"Falta columna en SPA normalizado: {req}")

    spa_df["spa_leg"] = spa_df["spa_leg"].astype(str).str.upper().str.strip()
    spa_df["RBD"]     = spa_df["RBD"].astype(str).str.upper().str.strip()

    # RTG numérico (alineado con rtg_map)
    spa_df["RTG"] = pd.to_numeric(spa_df["RTG"], errors="coerce")
    spa_df["RTG"] = spa_df["RTG"].round().astype("Int64")

    # rol por posición del endpoint en el leg
    spa_df["spa_start"] = spa_df["spa_leg"].str[:3]
    spa_df["spa_end"]   = spa_df["spa_leg"].str[-3:]

    agg = {
        "SPA_avg": ("SPA_avg", lambda x: round(pd.to_numeric(x, errors="coerce").mean(), 2)),
        "SPA_max": ("SPA_max", lambda x: round(pd.to_numeric(x, errors="coerce").max(), 2)),
        "SPA_legs_usados": ("spa_leg", lambda s: ",".join(sorted(s.astype(str).str.upper().unique()))),
    }
    if "SPA_min" in spa_df.columns:
        agg["SPA_min"] = ("SPA_min", lambda x: round(pd.to_numeric(x, errors="coerce").min(), 2))

    # DEST = el endpoint offline es el final del leg
    spa_dest_pick = (
        spa_df.groupby(["spa_end","RBD","RTG"], dropna=False)
              .agg(**agg).reset_index()
              .rename(columns={"spa_end":"OFFLINE_EP"})
    )
    # ORIG = el endpoint offline es el inicio del leg
    spa_orig_pick = (
        spa_df.groupby(["spa_start","RBD","RTG"], dropna=False)
              .agg(**agg).reset_index()
              .rename(columns={"spa_start":"OFFLINE_EP"})
    )
    return spa_dest_pick, spa_orig_pick



In [ ]:
# ================================================
# 🔹 1. Construir mapa RTG desde "Consolidado RTG"
# ================================================
rtg_map = build_rtg_map_from_consolidado(RTG_new)

# ================================================
# 🔹 2. Normalizar SPA y asignar RTG aplicable
# ================================================
spa_norm    = normalize_spa_columns(xf.parse(hoja_SPAs))
spa_with_rtg = attach_rtgs_to_spa(spa_norm, rtg_map)

print(f"SPA con RTG asignado: {spa_with_rtg.shape[0]} filas")
print(spa_with_rtg[["spa_leg","RBD","RTG"]].head())

# ================================================
# 🔹 3. Construir tablas SPA por rol (usa RTG)
# ================================================
spa_dest_pick, spa_orig_pick = preparar_spa_por_rol_con_rtg(spa_with_rtg)

# ================================================
# 🔹 4. Preparar tarifario: ORIG, DEST, OFFLINE_EP, OFFLINE_ROL
# ================================================
def endpoints_from_od_series(od_series: pd.Series):
    s = od_series.astype(str).str.strip().str.upper()
    return s.str[:3], s.str[-3:]

_df = df_filtrado_final.copy()
_df["__ord__"] = np.arange(len(_df))
# Routing numérico (mismo criterio que RTG)
_df["Routing number"] = pd.to_numeric(_df["Routing number"], errors="coerce")
_df["Routing number"] = _df["Routing number"].round().astype("Int64")

# RBD sí va como texto normalizado
_df["RBD"] = _df["RBD"].astype(str).str.upper().str.strip()

# Detecta endpoint offline y rol
orig, dest = endpoints_from_od_series(_df["OD"])
o_is_online = orig.isin(list(ENDPOINTS))
d_is_online = dest.isin(list(ENDPOINTS))
_df["OFFLINE_EP"]  = np.where(o_is_online & ~d_is_online, dest,
                       np.where(d_is_online & ~o_is_online, orig, pd.NA))
_df["OFFLINE_ROL"] = np.where(o_is_online & ~d_is_online, "DEST",
                       np.where(d_is_online & ~o_is_online, "ORIG", pd.NA))

# ================================================
# 🔹 5. Merge condicional por rol (llave: OFFLINE_EP, RBD, RTG)
# ================================================
mask_dest = _df["OFFLINE_ROL"].eq("DEST")
mask_orig = _df["OFFLINE_ROL"].eq("ORIG")

df_dest = _df[mask_dest].merge(
    spa_dest_pick,
    left_on=["OFFLINE_EP","RBD","Routing number"],
    right_on=["OFFLINE_EP","RBD","RTG"],
    how="left", validate="m:1"
).drop(columns=["RTG"], errors="ignore")

df_orig = _df[mask_orig].merge(
    spa_orig_pick,
    left_on=["OFFLINE_EP","RBD","Routing number"],
    right_on=["OFFLINE_EP","RBD","RTG"],
    how="left", validate="m:1"
).drop(columns=["RTG"], errors="ignore")

# ================================================
# 🔹 6. Filas no clasificables (ambos online u offline)
# ================================================
df_amb = _df[~(mask_dest | mask_orig)].copy()
for c in ["SPA_avg","SPA_max","SPA_min","SPA_legs_usados"]:
    if c not in df_amb.columns:
        df_amb[c] = pd.NA

# ================================================
# 🔹 7. Reconstrucción orden original
# ================================================
common_cols = set(_df.columns).union({"SPA_avg","SPA_max","SPA_legs_usados"})
if "SPA_min" in (set(df_dest.columns) | set(df_orig.columns)):
    common_cols.add("SPA_min")

df_dest = df_dest.reindex(columns=sorted(common_cols))
df_orig = df_orig.reindex(columns=sorted(common_cols))
df_amb  = df_amb.reindex(columns=sorted(common_cols))

frames = [d for d in (df_dest, df_orig, df_amb) if not d.empty]
df_con_spa = (pd.concat(frames, axis=0, ignore_index=False)
              if frames else _df.reindex(columns=sorted(common_cols)).copy())
df_con_spa = (df_con_spa
              .sort_values("__ord__", kind="stable")
              .drop(columns="__ord__", errors="ignore")
              .reset_index(drop=True))

# ================================================
# 🔹 8. Estado del match (mensajes actualizados a RTG)
# ================================================
df_con_spa["SPA_status"] = pd.NA
df_con_spa.loc[df_con_spa["OFFLINE_ROL"].isna(), "SPA_status"] = \
    "No clasificable (ambos online u offline)"

cond_no_spa = (
    df_con_spa["OFFLINE_ROL"].notna()
    & df_con_spa["SPA_avg"].isna()
    & df_con_spa["SPA_max"].isna()
)
df_con_spa.loc[cond_no_spa, "SPA_status"] = "Sin SPA para (OFFLINE_EP, RBD, RTG)"
df_con_spa.loc[~cond_no_spa & df_con_spa["OFFLINE_ROL"].notna(), "SPA_status"] = "OK"

print("✅ df_con_spa listo con merge por RTG")
print(df_con_spa[["OD","RBD","Routing number","OFFLINE_EP","SPA_avg","SPA_max","SPA_status"]].head())



SPA con RTG asignado: 55618 filas
  spa_leg RBD  RTG
0  FLLYQB   C  406
1  FLLYQB   J  406
2  FLLYQB   D  406
3  FLLYQB   F  406
4  FLLYQB   Y  406
✅ df_con_spa listo con merge por RTG
       OD RBD  Routing number OFFLINE_EP  SPA_avg  SPA_max SPA_status
0  ROCBOG   E             414        ROC    158.0      158         OK
1  ROCBOG   E             412        ROC     84.0       84         OK
2  ROCBOG   E             414        ROC    158.0      158         OK
3  ROCBOG   E             412        ROC     84.0       84         OK
4  ROCBOG   E             414        ROC    158.0      158         OK


In [ ]:
print("rtg_map RTG dtype:", rtg_map["RTG"].dtype)
print("spa_dest_pick RTG dtype:", spa_dest_pick["RTG"].dtype)
print("Tarifas Routing number dtype:", _df["Routing number"].dtype)

rtg_map RTG dtype: Int64
spa_dest_pick RTG dtype: Int64
Tarifas Routing number dtype: Int64


In [ ]:
df_con_spa.loc[df_con_spa["OD"].isin(["BOGDEN","SALDEN"])]

,Action code,Base price,Base_mas_Fuel,Bundle,Cabin,Currency,Destination,FN,Fareclass,OD,...,SPA_avg,SPA_legs_usados,SPA_max,SPA_min,Tarifa_USD,Tarifa_inicial_USD,Tariff code,Yield,kms,SPA_status


In [ ]:
df_con_spa.head(3)

,Action code,Base price,Base_mas_Fuel,Bundle,Cabin,Currency,Destination,FN,Fareclass,OD,...,SPA_avg,SPA_legs_usados,SPA_max,SPA_min,Tarifa_USD,Tarifa_inicial_USD,Tariff code,Yield,kms,SPA_status
0,I,638.0,715.0,B3,Y,USD,BOG,NaN,EHRB3BRS,ROCBOG,...,158.0,ROCWAS,158,158,638.0,1276.0,WHFP,16.682588,4285.905874,OK
1,I,817.5,894.5,B3,Y,USD,BOG,NaN,EHRB3BRS,ROCBOG,...,84.0,ROCCHI,84,84,817.5,1635.0,WHFP,20.870734,4285.905874,OK
2,I,485.0,562.0,B1,Y,USD,BOG,NaN,ELMB1BRS,ROCBOG,...,158.0,ROCWAS,158,158,485.0,485.0,WHFP,13.112747,4285.905874,OK


In [ ]:
# Verifica columnas detectadas
print("SPA norm cols:", spa_norm.columns.tolist())

# Muestra una muestra de los SPA con su RTG
print("spa_with_rtg sample:")
print(spa_with_rtg[["spa_leg","RBD","RTG"]].head())

# Prueba con la primera fila del tarifario
fila = _df.iloc[0]
print("\nFila ejemplo del tarifario:")
print(f"OD={fila['OD']}, RBD={fila['RBD']}, RTG={fila['Routing number']}, OFFLINE_EP={fila['OFFLINE_EP']}")

# Busca coincidencias en los SPA por RTG, RBD y OFFLINE_EP
match = spa_dest_pick[
    (spa_dest_pick["OFFLINE_EP"] == fila["OFFLINE_EP"]) &
    (spa_dest_pick["RBD"] == fila["RBD"]) &
    (spa_dest_pick["RTG"] == fila["Routing number"])
]
print("\nSPA que coinciden con la fila del tarifario:")
print(match.head())

SPA norm cols: ['spa_leg', 'RBD', 'SPA_avg', 'SPA_max', 'SPA_min', 'Partner']
spa_with_rtg sample:
  spa_leg RBD  RTG
0  FLLYQB   C  406
1  FLLYQB   J  406
2  FLLYQB   D  406
3  FLLYQB   F  406
4  FLLYQB   Y  406

Fila ejemplo del tarifario:
OD=ROCBOG, RBD=E, RTG=414, OFFLINE_EP=ROC

SPA que coinciden con la fila del tarifario:
      OFFLINE_EP RBD  RTG  SPA_avg  SPA_max SPA_legs_usados  SPA_min
13475        ROC   E  414    158.0      158          WASROC      158


In [ ]:
spa_with_rtg["RTG"].unique()

<IntegerArray>
[406, 416, 408, 411, 423, 418, 415, 420, 422, 425, 426, 412,   3,   5,   1,
 506, 409, 417, 421, 414, 424,   4, 428, 504,   2, 508, 500, 419]
Length: 28, dtype: Int64

In [ ]:
print("rtg_map cols:", rtg_map.columns.tolist())
print("rtg_map sample:\n", rtg_map.head())

spa_with_rtg = attach_rtgs_to_spa(spa_norm, rtg_map)
print("spa_with_rtg shape:", spa_with_rtg.shape)
print(spa_with_rtg[["spa_leg","RBD","RTG"]].head())

rtg_map cols: ['LegMetro', 'Partner', 'RTG']
rtg_map sample:
   LegMetro Partner  RTG
0   MADACE      IB    1
1   MADAGP      IB    1
2   MADALC      IB    1
3   MADBIO      IB    1
4   MADBJZ      IB    1
spa_with_rtg shape: (55618, 7)
  spa_leg RBD  RTG
0  FLLYQB   C  406
1  FLLYQB   J  406
2  FLLYQB   D  406
3  FLLYQB   F  406
4  FLLYQB   Y  406


In [ ]:
spa_with_rtg.head(5)

,spa_leg,RBD,SPA_avg,SPA_max,SPA_min,Partner,RTG
0,FLLYQB,C,610,610,610,AC,406
1,FLLYQB,J,582,582,582,AC,406
2,FLLYQB,D,504,504,504,AC,406
3,FLLYQB,F,504,504,504,AC,406
4,FLLYQB,Y,334,334,334,AC,406


In [ ]:
spa_with_rtg[spa_with_rtg["spa_leg"].str.contains("YTOYQM")]

,spa_leg,RBD,SPA_avg,SPA_max,SPA_min,Partner,RTG
4871,YTOYQM,C,419,547,291,AC,415
4872,YTOYQM,J,377,462,291,AC,415
4873,YTOYQM,D,362,433,291,AC,415
4874,YTOYQM,F,362,433,291,AC,415
4875,YTOYQM,Y,288,423,153,AC,415
4876,YTOYQM,B,288,423,153,AC,415
4877,YTOYQM,M,288,423,153,AC,415
4878,YTOYQM,H,276,398,153,AC,415
4879,YTOYQM,Q,222,291,153,AC,415
4880,YTOYQM,V,215,277,153,AC,415


In [ ]:
df_con_spa[['OFFLINE_EP','OFFLINE_ROL']]

,OFFLINE_EP,OFFLINE_ROL
0,ROC,ORIG
1,ROC,ORIG
2,ROC,ORIG
3,ROC,ORIG
4,ROC,ORIG
...,...,...
315,ROC,ORIG
316,ROC,ORIG
317,ROC,ORIG
318,ROC,ORIG


In [ ]:
df_con_spa.columns.tolist()

['Action code',
 'Base price',
 'Base_mas_Fuel',
 'Bundle',
 'Cabin',
 'Currency',
 'Destination',
 'FN',
 'Fareclass',
 'OD',
 'OFFLINE_EP',
 'OFFLINE_ROL',
 'OW/RT',
 'Origin',
 'RBD',
 'Routing number',
 'SPA_avg',
 'SPA_legs_usados',
 'SPA_max',
 'SPA_min',
 'Tarifa_USD',
 'Tarifa_inicial_USD',
 'Tariff code',
 'Yield',
 'kms',
 'SPA_status']

In [ ]:
## Filtrar df para solo usar las columnas necesarias para el calculo del yield after payout
cols_existentes=['Base price',               
 'Base_mas_Fuel',
 "Tarifa_inicial_USD",
 'Bundle',
 'Cabin',
 'Currency',
 "Origin",
 'Destination',
 "Action code",
 'FN',
 'Fareclass',
 "Routing number",
 'OD',
 'OW/RT',
 'RBD',
 'SPA_avg',
 'SPA_legs_usados',
 'SPA_max',
 'SPA_min',
 'Tarifa_USD',
 'kms',
 'SPA_status',
 "Yield",
 "OFFLINE_EP",
 "OFFLINE_ROL"]

df_yield_ap= df_con_spa[cols_existentes].copy()

# ====== CONFIG ======
kms_df = distancias 

od_col  = "OD"
kms_col = "KMS"

# Normaliza el df de distancias
_kms_map_df = kms_df[[od_col, kms_col]].copy()

# Diccionario leg -> kms
kms_lookup = dict(zip(_kms_map_df[od_col], _kms_map_df[kms_col]))

def _lookup_leg_kms(leg: str):
    """Devuelve kms del leg; si no existe, prueba con el leg invertido."""
    if not isinstance(leg, str):
        return np.nan
    L = leg.strip().upper()
    if not L or len(L) < 6:
        return np.nan
    val = kms_lookup.get(L, np.nan)
    if pd.isna(val):
        # intenta con invertido: ABCDEF -> DEFABC
        rev = L[-3:] + L[:3]
        val = kms_lookup.get(rev, np.nan)
    return val

def _split_legs(cell):
    if pd.isna(cell):
        return []
    # admite separadores coma/; y espacios
    return [x.strip() for x in str(cell).replace(";", ",").split(",") if x.strip()]

def _stats_for_legs(legs_list):
    if not legs_list:
        return (np.nan, np.nan, np.nan)
    vals = [ _lookup_leg_kms(leg) for leg in legs_list ]
    vals = [ v for v in vals if pd.notna(v) ]
    if not vals:
        return (np.nan, np.nan, np.nan)
    return (float(np.min(vals)), float(np.max(vals)), float(np.mean(vals)))

# Aplica al df_con_spa
legs_lists = df_yield_ap["SPA_legs_usados"].apply(_split_legs)
legs_lists = legs_lists.apply(lambda lst: [leg for leg in lst if "BOG" not in leg and "SAL" not in leg])
stats = legs_lists.apply(_stats_for_legs)

df_yield_ap["SPA_legs_kms_min"] = stats.apply(lambda t: t[0])
df_yield_ap["SPA_legs_kms_max"] = stats.apply(lambda t: t[1])
df_yield_ap["SPA_legs_kms_avg"] = stats.apply(lambda t: t[2])

# redondeo
DECIMALS = 0
for c in ["SPA_legs_kms_min", "SPA_legs_kms_max", "SPA_legs_kms_avg"]:
    df_yield_ap[c] = df_yield_ap[c].round(DECIMALS)

In [ ]:
# Añadir columna del ONLINE EP

df_yield_ap["Origin"] = df_yield_ap["Origin"].astype(str).str.upper().str.strip()
df_yield_ap["Destination"] = df_yield_ap["Destination"].astype(str).str.upper().str.strip()
df_yield_ap["OFFLINE_EP"] = df_yield_ap["OFFLINE_EP"].astype(str).str.upper().str.strip()

df_yield_ap["ONLINE_EP"] = np.where(
    df_yield_ap["OFFLINE_EP"] == df_yield_ap["Origin"],
    df_yield_ap["Destination"],
    df_yield_ap["Origin"]
)
def calc_kms_leg_online(row):
    spa_legs = row.get("SPA_legs_usados", np.nan)
    offline = str(row.get("OFFLINE_EP", "")).upper()
    online  = str(row.get("ONLINE_EP", "")).upper()

    if pd.isna(spa_legs) or spa_legs == "" or offline == "" or online == "":
        return np.nan

    dists = []

    # Ejemplo de formato esperado: "WASALB,NYCALB"
    for leg in str(spa_legs).split(","):
        leg = leg.strip().upper()
        if len(leg) != 6:
            # formato raro, lo saltamos
            continue

        a, b = leg[:3], leg[3:]  # p.ej. WASALB -> a='WAS', b='ALB'

        # Identificar cuál de los dos es el offline
        if a == offline:
            other = b
        elif b == offline:
            other = a
        else:
            # Si el offline no está en este leg, lo ignoramos
            continue

        # Construir leg online: ONLINE_EP + otro endpoint
        # Ej: online='GUA', other='WAS' -> 'GUAWAS'
        od_online = online + other

        kms_val = kms_lookup.get(od_online)
        if kms_val is not None:
            dists.append(kms_val)

    if dists:
        # promedio de las distancias encontradas
        return float(np.mean(dists))
    else:
        return np.nan

In [ ]:
df_yield_ap["KMs leg_online"] = df_yield_ap.apply(calc_kms_leg_online, axis=1)

In [ ]:
unique_legs = pd.Series([leg for legs in legs_lists for leg in legs]).unique()
print(unique_legs)

['ROCWAS' 'ROCCHI']


In [ ]:
df_yield_ap.head(10)

,Base price,Base_mas_Fuel,Tarifa_inicial_USD,Bundle,Cabin,Currency,Origin,Destination,Action code,FN,...,kms,SPA_status,Yield,OFFLINE_EP,OFFLINE_ROL,SPA_legs_kms_min,SPA_legs_kms_max,SPA_legs_kms_avg,ONLINE_EP,KMs leg_online
0,638.0,715.0,1276.0,B3,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,16.682588,ROC,ORIG,446.0,446.0,446.0,BOG,3808.558077
1,817.5,894.5,1635.0,B3,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,20.870734,ROC,ORIG,847.0,847.0,847.0,BOG,4363.724447
2,485.0,562.0,485.0,B1,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,13.112747,ROC,ORIG,446.0,446.0,446.0,BOG,3808.558077
3,720.0,797.0,720.0,B1,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,18.595835,ROC,ORIG,847.0,847.0,847.0,BOG,4363.724447
4,539.0,616.0,539.0,B2,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,14.372691,ROC,ORIG,446.0,446.0,446.0,BOG,3808.558077
5,775.0,852.0,775.0,B2,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,19.879111,ROC,ORIG,847.0,847.0,847.0,BOG,4363.724447
6,638.0,715.0,638.0,B3,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,16.682588,ROC,ORIG,446.0,446.0,446.0,BOG,3808.558077
7,870.0,947.0,870.0,B3,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,22.095679,ROC,ORIG,847.0,847.0,847.0,BOG,4363.724447
8,435.0,512.0,870.0,B1,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,11.946133,ROC,ORIG,446.0,446.0,446.0,BOG,3808.558077
9,617.5,694.5,1235.0,B1,Y,USD,ROC,BOG,I,NaN,...,4285.905874,OK,16.204276,ROC,ORIG,847.0,847.0,847.0,BOG,4363.724447


In [ ]:
df_yield_ap[df_yield_ap["SPA_legs_usados"]=="YTOYQM"]

,Base price,Base_mas_Fuel,Tarifa_inicial_USD,Bundle,Cabin,Currency,Origin,Destination,Action code,FN,...,kms,SPA_status,Yield,OFFLINE_EP,OFFLINE_ROL,SPA_legs_kms_min,SPA_legs_kms_max,SPA_legs_kms_avg,ONLINE_EP,KMs leg_online


In [ ]:
df_yield_ap.columns.tolist()

['Base price',
 'Base_mas_Fuel',
 'Tarifa_inicial_USD',
 'Bundle',
 'Cabin',
 'Currency',
 'Origin',
 'Destination',
 'Action code',
 'FN',
 'Fareclass',
 'Routing number',
 'OD',
 'OW/RT',
 'RBD',
 'SPA_avg',
 'SPA_legs_usados',
 'SPA_max',
 'SPA_min',
 'Tarifa_USD',
 'kms',
 'SPA_status',
 'Yield',
 'OFFLINE_EP',
 'OFFLINE_ROL',
 'SPA_legs_kms_min',
 'SPA_legs_kms_max',
 'SPA_legs_kms_avg',
 'ONLINE_EP',
 'KMs leg_online']

In [ ]:
df_yield_ap.loc[df_yield_ap["OD"].isin(["BOGDEN", "SALDEN"]),["OFFLINE_EP","OFFLINE_ROL","SPA_legs_usados"]]

,OFFLINE_EP,OFFLINE_ROL,SPA_legs_usados


In [ ]:
ods_sin_spa = (
    df_yield_ap.loc[df_yield_ap["SPA_legs_usados"].isna(), "OD"]
    .dropna()
    .drop_duplicates()
    .sort_values()
    .tolist()
)

print(f"ODS sin SPA asignado: {len(ods_sin_spa)}")
print(ods_sin_spa)

ODS sin SPA asignado: 0
[]


In [ ]:
# ODs problemáticos
ods_problema = ["SALSAN","SANSAL"]
tmp = df_yield_ap.loc[df_yield_ap["OD"].isin(ods_problema),
                      ["OD","RBD","Routing number","OFFLINE_EP","OFFLINE_ROL","SPA_legs_usados","SPA_status"]]
print("Filas del tarifario afectadas:")
print(tmp.drop_duplicates())

# Por cada OD, inspecciona qué hay en spa_dest_pick / spa_orig_pick con ese RTG + EP
for _, r in tmp.drop_duplicates().iterrows():
    od   = str(r["OD"])
    rtg  = str(r["Routing number"]).upper().strip()
    rbd  = str(r["RBD"]).upper().strip()
    ep   = str(r["OFFLINE_EP"]).upper().strip()
    rol  = str(r["OFFLINE_ROL"]).upper().strip()

    print("\n=== OD:", od, "| RTG:", rtg, "| RBD:", rbd, "| EP:", ep, "| ROL:", rol, "===")
    if rol == "DEST":
        cand = spa_dest_pick
    elif rol == "ORIG":
        cand = spa_orig_pick
    else:
        cand = None

    if cand is None:
        print("No clasificable (ROL vacío), por eso no hay SPA.")
        continue

    # 1) ¿Hay algo con ese RTG?
    print("• Picks por RTG:")
    print(cand.loc[cand["RTG"].astype(str).str.upper().eq(rtg),
                   ["OFFLINE_EP","RBD","RTG","SPA_legs_usados"]].drop_duplicates().head(20))

    # 2) ¿Y con ese RTG + EP?
    print("• Picks por RTG + EP:")
    print(cand.loc[
        cand["RTG"].astype(str).str.upper().eq(rtg) &
        cand["OFFLINE_EP"].astype(str).str.upper().eq(ep),
        ["OFFLINE_EP","RBD","RTG","SPA_legs_usados"]].drop_duplicates().head(20))

    # 3) ¿Y con RTG + EP + RBD?
    print("• Picks por RTG + EP + RBD:")
    print(cand.loc[
        cand["RTG"].astype(str).str.upper().eq(rtg) &
        cand["OFFLINE_EP"].astype(str).str.upper().eq(ep) &
        cand["RBD"].astype(str).str.upper().eq(rbd),
        ["OFFLINE_EP","RBD","RTG","SPA_legs_usados","SPA_avg","SPA_max","SPA_min"]].head(20))

# Además, verifica todos los legs con 'SAN' que existan en spa_with_rtg y qué RTG tienen
print("\nTodos los legs con 'SAN' en spa_with_rtg (muestra):")
print(
    spa_with_rtg.loc[spa_with_rtg["spa_leg"].str.contains("SAN", na=False),
                     ["spa_leg","RBD","RTG"]]
    .drop_duplicates()
    .head(50)
)



Filas del tarifario afectadas:
Empty DataFrame
Columns: [OD, RBD, Routing number, OFFLINE_EP, OFFLINE_ROL, SPA_legs_usados, SPA_status]
Index: []

Todos los legs con 'SAN' en spa_with_rtg (muestra):
     spa_leg RBD  RTG
2057  SANYMQ   C  423
2058  SANYMQ   J  423
2059  SANYMQ   D  423
2060  SANYMQ   F  423
2061  SANYMQ   Y  423
2062  SANYMQ   B  423
2063  SANYMQ   M  423
2064  SANYMQ   H  423
2065  SANYMQ   Q  423
2066  SANYMQ   V  423
2067  SANYMQ   A  423
2068  SANYMQ   E  423
2069  SANYMQ   K  423
2070  SANYMQ   L  423
2071  SANYMQ   O  423
2072  SANYMQ   P  423
2073  SANYMQ   Z  423
2074  SANYMQ   W  423
2075  SANYMQ   S  423
2076  SANYMQ   T  423
2077  SANYMQ   U  423
2078  SANYTO   C  415
2079  SANYTO   J  415
2080  SANYTO   D  415
2081  SANYTO   F  415
2082  SANYTO   Y  415
2083  SANYTO   B  415
2084  SANYTO   M  415
2085  SANYTO   H  415
2086  SANYTO   Q  415
2087  SANYTO   V  415
2088  SANYTO   A  415
2089  SANYTO   E  415
2090  SANYTO   K  415
2091  SANYTO   L  415
2092  SAN

In [ ]:
df_yield_ap[df_yield_ap["SPA_legs_usados"].isna()]

,Base price,Base_mas_Fuel,Tarifa_inicial_USD,Bundle,Cabin,Currency,Origin,Destination,Action code,FN,...,kms,SPA_status,Yield,OFFLINE_EP,OFFLINE_ROL,SPA_legs_kms_min,SPA_legs_kms_max,SPA_legs_kms_avg,ONLINE_EP,KMs leg_online


In [ ]:
df_yield_ap["SPA_legs_usados"].value_counts()

SPA_legs_usados
ROCWAS    160
ROCCHI    160
Name: count, dtype: int64

## Incluir nueva hoja de SPAs vacios al excel

In [ ]:
# ====== 1) Selección y orden de columnas ======
cols_wanted = [
    "OD",
    "RBD",
    "Routing number",
    "SPA_legs_usados",
    "SPA_min",
    "SPA_max",
    "Tarifa_USD",
    "kms",
    "SPA_status",
    "Yield",
    "SPA_legs_kms_min",
    "SPA_legs_kms_max",
    "SPA_legs_kms_avg",
]

# Mantén solo las columnas que existan en df_yield_ap y respeta el orden deseado
cols_final = [c for c in cols_wanted if c in df_yield_ap.columns]
spa_detalle = df_yield_ap.loc[:, cols_final].copy()

# Conteo de legs y flag vacío
def _split_legs(cell):
    if pd.isna(cell):
        return []
    return [x.strip() for x in str(cell).replace(";", ",").split(",") if x.strip()]

spa_detalle["#legs"] = spa_detalle["SPA_legs_usados"].apply(_split_legs).str.len() if "SPA_legs_usados" in spa_detalle.columns else pd.NA
spa_detalle["legs_vacios"] = (spa_detalle["#legs"]==0) | (spa_detalle["#legs"].isna()) if "#legs" in spa_detalle.columns else pd.NA

# Coerce a numéricos (por si llegan como string)
for c in ["SPA_min","SPA_max","Tarifa_USD","kms","Yield",
          "SPA_legs_kms_min","SPA_legs_kms_max","SPA_legs_kms_avg"]:
    if c in spa_detalle.columns:
        spa_detalle[c] = pd.to_numeric(spa_detalle[c], errors="coerce")

# ====== 3) Diagnóstico de filas con SPA_legs_usados vacío ======
if "SPA_legs_usados" in spa_detalle.columns:
    mask_vacios = spa_detalle["legs_vacios"] == True
    spa_vacios = spa_detalle.loc[mask_vacios].copy()
else:
    spa_vacios = pd.DataFrame()

# ====== 4) Exportar a Excel con tu helper ======
#   - Hoja 1: detalle completo
#   - Hoja 2: únicamente los vacíos (si hay)
write_sheet(spa_detalle, excel_path, sheet_name="SPA legs (detalle)", replace=True)

if not spa_vacios.empty:
    write_sheet(spa_vacios, excel_path, sheet_name="SPA legs (VACIOS)", replace=True)
else:
    # Si prefieres no escribir la hoja cuando no hay vacíos, comenta la línea de abajo
   write_sheet(spa_vacios, excel_path, sheet_name="SPA legs (VACIOS)", replace=True)

print("Hojas creadas en:", excel_path)

Hojas creadas en: C:\Users\amvillamizar\Downloads\Avianca\Offline\Documentos\Publicacion tarifas\Pricing_checks\Parametros_Offline\Resumen\140126_PRUEBA_ABS.xlsx


In [ ]:
df_yield_ap["dist_min_payout"] = (df_yield_ap["KMs leg_online"]+df_yield_ap["SPA_legs_kms_avg"])- df_yield_ap["SPA_legs_kms_max"]

df_yield_ap["dist_max_payout"] = (df_yield_ap["KMs leg_online"]+df_yield_ap["SPA_legs_kms_avg"])- df_yield_ap["SPA_legs_kms_min"]

df_yield_ap["dist_avg_payout"] = df_yield_ap["KMs leg_online"]

df_yield_ap["Prorrateo"] = df_yield_ap["Base_mas_Fuel"] * (df_yield_ap["SPA_legs_kms_avg"] /(df_yield_ap["SPA_legs_kms_avg"] + df_yield_ap["KMs leg_online"]))

In [ ]:
df_yield_ap["yield_min_payout"] =(df_yield_ap["Base_mas_Fuel"] - df_yield_ap["SPA_min"] )*100/df_yield_ap["dist_min_payout"]
df_yield_ap["yield_max_payout"] =(df_yield_ap["Base_mas_Fuel"] - df_yield_ap["SPA_max"] )*100/df_yield_ap["dist_max_payout"]
df_yield_ap["yield_avg_payout"] =(df_yield_ap["Base_mas_Fuel"] - df_yield_ap[["SPA_avg","Prorrateo"]].max(axis=1) )*100/df_yield_ap["dist_avg_payout"]
df_yield_ap["B+F aft payout"] = df_yield_ap["Base_mas_Fuel"] - df_yield_ap[["SPA_avg","Prorrateo"]].max(axis=1)


In [ ]:
df_yield_ap["B+F aft max payout"] = df_yield_ap["Base_mas_Fuel"] - df_yield_ap[["SPA_max","Prorrateo"]].max(axis=1)
df_yield_ap["B+F aft min payout"] = df_yield_ap["Base_mas_Fuel"] - df_yield_ap[["SPA_min","Prorrateo"]].max(axis=1)

In [ ]:
df_yield_ap[["Base_mas_Fuel","SPA_legs_kms_avg","kms","Prorrateo","dist_min_payout","dist_max_payout","dist_avg_payout","Yield","yield_avg_payout","yield_min_payout","yield_max_payout","SPA_avg"]].head(10)

,Base_mas_Fuel,SPA_legs_kms_avg,kms,Prorrateo,dist_min_payout,dist_max_payout,dist_avg_payout,Yield,yield_avg_payout,yield_min_payout,yield_max_payout,SPA_avg
0,715.0,446.0,4285.905874,74.952555,3808.558077,3808.558077,3808.558077,16.682588,14.624957,14.624957,14.624957,158.0
1,894.5,847.0,4285.905874,145.400416,4363.724447,4363.724447,4363.724447,20.870734,17.166519,18.573583,18.573583,84.0
2,562.0,446.0,4285.905874,58.913757,3808.558077,3808.558077,3808.558077,13.112747,10.607689,10.607689,10.607689,158.0
3,797.0,847.0,4285.905874,129.551852,4363.724447,4363.724447,4363.724447,18.595835,15.295378,16.339254,16.339254,84.0
4,616.0,446.0,4285.905874,64.574509,3808.558077,3808.558077,3808.558077,14.372691,12.025549,12.025549,12.025549,158.0
5,852.0,847.0,4285.905874,138.492067,4363.724447,4363.724447,4363.724447,19.879111,16.350893,17.599645,17.599645,84.0
6,715.0,446.0,4285.905874,74.952555,3808.558077,3808.558077,3808.558077,16.682588,14.624957,14.624957,14.624957,158.0
7,947.0,847.0,4285.905874,153.934258,4363.724447,4363.724447,4363.724447,22.095679,18.174056,19.776684,19.776684,84.0
8,512.0,446.0,4285.905874,53.672319,3808.558077,3808.558077,3808.558077,11.946133,9.294856,9.294856,9.294856,158.0
9,694.5,847.0,4285.905874,112.890541,4363.724447,4363.724447,4363.724447,16.204276,13.328281,13.990343,13.990343,84.0


In [ ]:
df_yield_ap["B+F aft payout"].head(10)

0    557.000000
1    749.099584
2    404.000000
3    667.448148
4    458.000000
5    713.507933
6    557.000000
7    793.065742
8    354.000000
9    581.609459
Name: B+F aft payout, dtype: float64

In [ ]:
# Filtramos las filas donde yield_avg_payout < 1.5
df_filtrado = df_yield_ap.loc[df_yield_ap["yield_min_payout"] < 1.5]

# Exportamos a Excel
df_filtrado.to_excel("filtrado_yield_avg_payout_menor_1_5.xlsx", index=False)


In [ ]:
# --- Resumen OD / Routing / RBD usando columnas ya creadas en df_yield_ap ---
# Columnas requeridas en df_yield_ap: "Base_mas_Fuel", "SPA_min", "SPA_max", "SPA_avg", "Prorrateo"
# Mostrar además SÓLO: "B+F aft max payout" y "B+F aft min payout" (si existen)

src = df_yield_ap.copy()

group_cols = ["OD", "Routing number", "RBD"]
for c in group_cols:
    if c not in src.columns:
        src[c] = pd.NA

# Forzar numéricos en las columnas que necesitamos
for c in ["Base_mas_Fuel", "SPA_min", "SPA_max", "SPA_avg", "Prorrateo"]:
    if c in src.columns:
        src[c] = pd.to_numeric(src[c], errors="coerce")

# Agrupar y calcular
grp = src.groupby(group_cols, dropna=False)
agg_df = pd.DataFrame({"Base_mas_Fuel_min": grp["Base_mas_Fuel"].min()}).reset_index()
if "SPA_max" in src.columns:
    agg_df["SPA_MAX"] = grp["SPA_max"].max().values
if "SPA_avg" in src.columns:
    agg_df["SPA_AVG"] = grp["SPA_avg"].mean().values
if "SPA_min" in src.columns:
    agg_df["SPA_MIN"] = grp["SPA_min"].min().values

# Localizar fila con Base_mas_Fuel mínima para traer Prorrateo y payouts
idx_min = grp["Base_mas_Fuel"].idxmin().dropna().reset_index()
idx_min.columns = group_cols + ["_idx"]

payout_cols = ["B+F aft max payout", "B+F aft min payout"]
select_cols = group_cols + ["Base_mas_Fuel", "Prorrateo"] + [c for c in payout_cols if c in src.columns]

if not idx_min.empty:
    min_rows = src.loc[idx_min["_idx"].astype(int)].reset_index(drop=True)
    min_rows[group_cols] = idx_min[group_cols].values
    min_rows_sel = min_rows[[c for c in select_cols if c in min_rows.columns]].copy()
    if "Base_mas_Fuel" in min_rows_sel.columns:
        min_rows_sel = min_rows_sel.rename(columns={"Base_mas_Fuel": "Base_mas_Fuel_at_min"})
    final = agg_df.merge(min_rows_sel, on=group_cols, how="left")
else:
    final = agg_df.copy()

# Redondeo de columnas numéricas relevantes
for c in ["Base_mas_Fuel_min", "Base_mas_Fuel_at_min", "Prorrateo", "B+F aft max payout", "B+F aft min payout", "SPA_AVG"]:
    if c in final.columns:
        final[c] = pd.to_numeric(final[c], errors="coerce").round(2)

# Reordenar columnas para salida
cols_order = group_cols + ["Base_mas_Fuel_min", "Base_mas_Fuel_at_min", "SPA_MAX", "SPA_AVG", "SPA_MIN", "Prorrateo"] + [c for c in payout_cols if c in final.columns]
cols_present = [c for c in cols_order if c in final.columns]
final = final.loc[:, cols_present]

# Mostrar verificación
print("Columnas incluidas en el resumen:", final.columns.tolist())
print("Muestra (5 filas):")
print(final.head(5).to_string(index=False))

# Escribir a Excel
try:
    write_sheet(final, excel_path, sheet_name="OD_Routing_Summary", replace=True)
    print(f"✅ Hoja 'OD_Routing_Summary' escrita en: {excel_path}")
except PermissionError:
    print(f"⚠️ PermissionError: no pude escribir {excel_path}. Asegúrate de que el archivo esté cerrado y vuelve a ejecutar esta celda.")
except Exception as e:
    print("Error al escribir la hoja:", e)

Columnas incluidas en el resumen: ['OD', 'Routing number', 'RBD', 'Base_mas_Fuel_min', 'Base_mas_Fuel_at_min', 'SPA_MAX', 'SPA_AVG', 'SPA_MIN', 'Prorrateo', 'B+F aft max payout', 'B+F aft min payout']
Muestra (5 filas):
    OD Routing number RBD  Base_mas_Fuel_min  Base_mas_Fuel_at_min  SPA_MAX  SPA_AVG  SPA_MIN  Prorrateo  B+F aft max payout  B+F aft min payout
ROCBOG            412   A              882.0                 882.0       84     84.0       84     143.37              738.63               798.0
ROCBOG            412   B             2582.0                2582.0       84     84.0       84     419.70             2162.30              2498.0
ROCBOG            412   E              694.5                 694.5       84     84.0       84     112.89              581.61               610.5
ROCBOG            412   H             1762.0                1762.0       84     84.0       84     286.41             1475.59              1678.0
ROCBOG            412   K              544.5           

### Si se van a subir una nueva estructura, verificar que esten todos los bundles

In [ ]:
# ========= 1) CONFIG =========
COL = "Action code"
MAP = {"N": "N", "C": "X", "I/R": ["I", "R", ""]}  # Criterios por opción
df=df_yield_ap.copy()

def elegir_opcion_action_code():
    # Devuelve "N" | "C" | "I/R" o None si se cancela
    eleccion = {"val": None}

    root = tk.Tk()
    root.withdraw()  # no muestres la raíz

    win = tk.Toplevel(root)
    win.title("Seleccionar si el request pasa por New, Cancel o Increment/Reduction")
    # ventanita centrada y siempre al frente
    try:
        win.attributes("-topmost", True)
    except Exception:
        pass
    win.geometry("340x180")

    # callbacks
    def set_val(v):
        eleccion["val"] = v
        try:
            win.destroy()
        finally:
            root.destroy()

    def on_close():
        # cancelar explícitamente
        eleccion["val"] = None
        try:
            win.destroy()
        finally:
            root.destroy()

    # contenido
    tk.Label(win, text="¿Qué tipo de request se está revisando?").pack(pady=10)
    for opt in ["N", "C", "I/R"]:
        tk.Button(win, text=opt, command=lambda v=opt: set_val(v)).pack(fill="x", padx=20, pady=6)

    # accesos rápidos: Enter = primera opción, Esc = cancelar
    win.bind("<Return>", lambda e: set_val("N"))
    win.bind("<Escape>", lambda e: on_close())
    win.protocol("WM_DELETE_WINDOW", on_close)

    # foco y exclusividad
    win.deiconify()
    win.lift()
    win.focus_force()
    win.grab_set()

    # en notebooks, ayuda tener %gui tk; aquí esperamos a que cierre
    root.wait_window(win)
    return eleccion["val"]

# ========= 3) FUNCIÓN: construir líneas para el TXT (VALIDACIÓN) =========
def build_selection_validation_lines(df, col, opcion, titulo="Validación por selección de usuario"):
    lines = [f"{titulo}\n"]

    # sanity checks
    if opcion not in MAP:
        lines.append("❌ Opción inválida; no se ejecutó esta validación.\n\n")
        return lines
    if col not in df.columns:
        lines.append(f"❌ La columna '{col}' no existe en el DataFrame.\n\n")
        return lines

    # normaliza a string, mayúsculas; NaN -> "" (para poder aceptar vacío)
    s = df[col].astype("string").str.strip().str.upper().fillna("")

    # asegura lista de esperados
    vals = MAP[opcion]
    esperados = [vals.strip().upper()] if isinstance(vals, str) else [str(v).strip().upper() for v in vals]

    # chequeo
    ok_mask = s.isin(esperados)
    passed = bool(ok_mask.all())
    mismatches = int((~ok_mask).sum())

    if passed:
        if opcion == "I/R":
            lines.append(f"✅ Opción {opcion}: todas las filas de '{col}' están en {{I, R, vacío}}.\n\n")
        else:
            lines.append(f"✅ Opción {opcion}: todas las filas de '{col}' son '{esperados[0]}'.\n\n")
    else:
        invalid_vals = (
            s[~ok_mask]
            .astype(str)
            .replace({"": "(vacío)"})
            .value_counts()
            .head(15)
        )

        def _bullets_from_vc(vc):
            if vc.empty:
                return "- (ninguno)\n"
            return "".join([f"- {idx}: {cnt}\n" for idx, cnt in vc.items()])

        if opcion == "I/R":
            lines.append(f"⚠️ Opción {opcion}: {mismatches} filas NO están en {{I, R, vacío}} para la columna '{col}'.\n")
        else:
            lines.append(f"⚠️ Opción {opcion}: {mismatches} filas NO son '{esperados[0]}' para la columna '{col}'.\n")
        lines.append("Valores no permitidos (muestra, valor: conteo):\n")
        lines.append(_bullets_from_vc(invalid_vals) + "\n")

    return lines


## Validacion RBDs

## Chequeo threshold minimo tarifas OW/RT

In [ ]:
def build_baseprice_check_lines(
    df,
    col_price="Tarifa_USD",
    col_owrt="OW/RT",
    col_od="OD",
    col_fc="Fareclass",
    titulo="7. Validación Base_price por OW/RT"
):
    lines = [f"{titulo}\n"]
    required = [col_price, col_owrt, col_od, col_fc]
    missing = [c for c in required if c not in df.columns]
    if missing:
        lines.append(f"❌ Faltan columnas en el DataFrame: {', '.join(missing)}\n\n")
        return lines

    # Asegurar numérico en Base_price
    dfc = df_yield_ap.copy()
    dfc[col_price] = pd.to_numeric(dfc[col_price], errors="coerce")

    # Máscaras de incumplimiento
    mask_ow1_fail = (dfc[col_owrt] == 1) & (dfc[col_price] < 80)
    mask_ow2_fail = (dfc[col_owrt] == 2) & (dfc[col_price] <160 )

    fails = dfc[mask_ow1_fail | mask_ow2_fail].copy()
    n_fail = len(fails)
    tot = len(dfc)

    if n_fail == 0:
        lines.append(
            f"✅ Todas las {tot} tarifas cumplen las reglas de Base_price "
            f"(>=80 si OW/RT=1, >=160 si OW/RT=2).\n\n"
        )
    else:
        lines.append(
            f"⚠️ {n_fail} de {tot} tarifas NO cumplen las reglas de Base_price (OW>80 y RT>160).\n"
        )
        # muestra de ejemplos con OD y Fareclass
        muestra = fails[[col_od, col_fc, col_price, col_owrt]].head(20)

        # convierto en tabla Markdown
        out = "| OD | Fareclass | Base_price | OW/RT |\n|---|---|---|---|\n"
        for _, r in muestra.iterrows():
            out += (
                f"| {r[col_od]} | {r[col_fc]} | {r[col_price]} | {r[col_owrt]} |\n"
            )
        if n_fail > 20:
            out += f"\n… ({n_fail-20} filas más)\n"

        lines.append("Ejemplos de tarifas no válidas:\n\n")
        lines.append(out + "\n")

    return lines

## Chequeo tarifas OW>RT

In [ ]:
def build_owrt_tarifa_check_lines(
    df, 
    col_od="OD", 
    col_rtg="Routing number", 
    col_rbd="RBD", 
    col_bundle="Bundle",
    col_flag="OW/RT", 
    col_tarifa="Base_mas_Fuel", 
    col_fc="Fareclass",
    col_cabin="Cabin",           # 🔹 Nuevo parámetro
    cabin_value="Y",             # 🔹 Solo procesar si Cabin == "Y"
    writer=None,   
    titulo="8. Validación tarifa OW > RT (comparando por carácter 2 del Fareclass)"
):
    lines = []
    lines.append(titulo + "\n")
    lines.append("-" * len(titulo) + "\n")

    # ---- Filtrar solo filas con Cabin == cabin_value ----
    df = df.copy()
    df = df[df[col_cabin] == cabin_value]

    if df.empty:
        lines.append(f"⚠️ No existen filas con Cabin = '{cabin_value}', no se realiza validación.\n\n")
        return lines

    # ---- Normalizaciones básicas ----
    df[col_flag] = pd.to_numeric(df[col_flag], errors="coerce").astype("Int64")

    # FC2 = segundo carácter del Fareclass
    if col_fc in df.columns:
        df[col_fc] = df[col_fc].astype(str).str.strip().str.upper()
        df["H/L"] = df[col_fc].str[1].fillna("(NA)")
    else:
        df["H/L"] = "(NA)"

    # ---- Pivot ----
    index_cols = [col_od, col_rtg, col_rbd, col_bundle, "H/L"]

    pivot = (
        df.pivot_table(
            index=index_cols,
            columns=col_flag,           # 1=OW, 2=RT
            values=col_tarifa,
            aggfunc="first"
        )
        .reindex(columns=[1, 2], fill_value=np.nan)
        .rename(columns={1: "tarifa_ow", 2: "tarifa_rt"})
        .reset_index()
    )

    # Presencia de ambas tarifas
    pivot["has_ow"] = pivot["tarifa_ow"].notna()
    pivot["has_rt"] = pivot["tarifa_rt"].notna()
    pivot["ambos"]  = pivot["has_ow"] & pivot["has_rt"]

    # Validación OW > RT
    pivot["OK"]   = np.where(pivot["ambos"], pivot["tarifa_ow"] > pivot["tarifa_rt"], np.nan)
    pivot["diff"] = pivot["tarifa_ow"] - pivot["tarifa_rt"]

    # ---- Filtrar violaciones ----
    violaciones = pivot[pivot["ambos"] & (~pivot["OK"].astype(bool))].copy()

    # Caso sin errores
    if violaciones.empty:
        lines.append(
            f"✅ Todas las combinaciones (Cabin = '{cabin_value}') cumplen la condición OW > RT.\n\n"
        )
        return lines

    # Caso con errores
    lines.append(f"❌ {len(violaciones)} combinaciones NO cumplen (solo Cabin = '{cabin_value}'):\n")
    for _, row in violaciones.head(10).iterrows():
        lines.append(
            f"- OD={row[col_od]} | RTG={row[col_rtg]} | RBD={row[col_rbd]} | "
            f"Bundle={row[col_bundle]} | H/L={row['H/L']} | diff(OW-RT)={row['diff']:.2f}\n"
        )
    lines.append("\n")

    # ---- Exportar detalle ----
    if writer is not None:
        violaciones.to_excel(writer, sheet_name="Errores_OWvsRT", index=False)

    return lines


In [ ]:
# ======= Config columnas =======
COL_CABIN     = "Cabin"
COL_RBD       = "RBD"
COL_TARIFF    = "Tariff code"      # para contar # TARIFAS (nunique o filas)
COL_YIELD     = "Yield"            # mínimo de yield (pre-payout)
COL_BQ_OW     = "Base_mas_Fuel"    # mínimo de Base+Fuel OW en USD
COL_FARECLASS = "Fareclass"        # <-- NUEVO: para reportar el fareclass del min B+F
COL_OD        = "OD"               # para conteo de tarifas únicas
COL_BQ_AP     = "B+F aft payout"         # mínimo de Base+Fuel after payout (opcional)

# ======= 1) Mapeo explícito letra -> bucket =======
rbd_to_bucket = {
    "U": "U-T", "T": "U-T",
    "S": "S",
    "W": "W-O", "Z": "W-O", "P": "W-O", "O": "W-O",
    "L": "L-A", "K": "L-A", "E": "L-A", "A": "L-A",
    "V": "V-M", "Q": "V-M", "H": "V-M", "M": "V-M",
    "B": "B-Y", "Y": "B-Y",
    "F": "F",
    "D": "D-J", "J": "D-J",
    "C": "C",
}

# ======= 2) Buckets por Cabin (orden exacto del reporte) =======
buckets_por_cabin = {
    "Y": ["U-T", "S", "W-O", "L-A", "V-M", "B-Y"],
    "C": ["F", "D-J", "C"],
}

# ======= 3) Origen de datos =======
df = df_yield_ap.copy()

# Normalización básica
df[COL_CABIN]     = df[COL_CABIN].astype(str).str.strip().str.upper()
df[COL_RBD]       = df[COL_RBD].astype(str).str.strip().str.upper().str[0]
df[COL_YIELD]     = pd.to_numeric(df.get(COL_YIELD, pd.NA), errors="coerce")
df[COL_BQ_OW]     = pd.to_numeric(df.get(COL_BQ_OW, pd.NA), errors="coerce")
df[COL_BQ_AP]     = pd.to_numeric(df.get(COL_BQ_AP, pd.NA), errors="coerce")

if COL_FARECLASS in df.columns:
    df[COL_FARECLASS] = df[COL_FARECLASS].astype(str).str.strip().str.upper()   # <-- opcional

# Asegura columnas SPA y distancias como numéricas (si existen)
for c in ["SPA_min","SPA_max","SPA_avg",
          "dist_min_payout","dist_max_payout","dist_avg_payout",
          "SPA_legs_kms_min","SPA_legs_kms_max","SPA_legs_kms_avg"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

def _safe_yield(base, spa, dist):
    base = pd.to_numeric(base, errors="coerce")
    spa  = pd.to_numeric(spa,  errors="coerce")
    dist = pd.to_numeric(dist, errors="coerce")
    y = (base - spa) * 100 / dist
    y[(dist<=0) | dist.isna()] = pd.NA
    return y

# Asigna bucket según el mapeo explícito
df["RBD_bucket"] = df[COL_RBD].map(rbd_to_bucket)

# (Opcional) Diagnóstico de RBD sin mapeo
sin_mapeo = df.loc[df["RBD_bucket"].isna(), COL_RBD].value_counts()
if not sin_mapeo.empty:
    print("⚠️ RBD sin mapeo encontrado (revisa 'rbd_to_bucket'):\n", sin_mapeo.to_string())

# ======= 4) Construir el resumen en el orden deseado =======
filas_resumen = []
for cabin, buckets in buckets_por_cabin.items():
    for bucket in buckets:
        sub = df[(df[COL_CABIN] == cabin) & (df["RBD_bucket"] == bucket)]
        n_filas       = len(sub)

        # Mínimos / máximos por bucket (pre-payout)
        min_yield_pre = sub[COL_YIELD].min(skipna=True)
        min_bq        = sub[COL_BQ_OW].min(skipna=True)
        max_bq        = sub[COL_BQ_OW].max(skipna=True)

        # Mínimos / máximos por bucket (aft payout)
        if COL_BQ_AP in sub.columns:
            min_bq_ap = sub[COL_BQ_AP].min(skipna=True)
            max_bq_ap = sub[COL_BQ_AP].max(skipna=True)
        else:
            min_bq_ap = pd.NA
            max_bq_ap = pd.NA

        # Índice de la fila con el mínimo B+F after payout
        fareclass_min_ap = None
        od_min_ap        = None
        if n_filas > 0 and COL_BQ_AP in sub.columns and sub[COL_BQ_AP].notna().any():
            idx_min_ap = sub[COL_BQ_AP].idxmin()
            if pd.notna(idx_min_ap):
                if COL_FARECLASS in sub.columns:
                    fareclass_min_ap = sub.loc[idx_min_ap, COL_FARECLASS]
                if COL_OD in sub.columns:
                    od_min_ap = sub.loc[idx_min_ap, COL_OD]

        # Yield after payout (mínimos por bucket si existen)
        min_yield_min_payout = sub["yield_min_payout"].min(skipna=True) if "yield_min_payout" in sub.columns else pd.NA
        min_yield_max_payout = sub["yield_max_payout"].min(skipna=True) if "yield_max_payout" in sub.columns else pd.NA
        min_yield_avg_payout = sub["yield_avg_payout"].min(skipna=True) if "yield_avg_payout" in sub.columns else pd.NA

        filas_resumen.append({
            "Cabin": cabin,
            "RBD": bucket,
            "Tarifas": n_filas,

            # Mínimos/máximos B+F pre / aft payout
            "Min B+F": (None if pd.isna(min_bq) else min_bq),
            "B+F aft payout": (None if pd.isna(min_bq_ap) else min_bq_ap),

            "Max B+F": (None if pd.isna(max_bq) else max_bq),
            "Max B+F aft payout": (None if pd.isna(max_bq_ap) else max_bq_ap),

            # OD / Fareclass asociados al mínimo B+F after payout
            "OD": (None if pd.isna(od_min_ap) else od_min_ap),
            "Fareclass": (None if pd.isna(fareclass_min_ap) else fareclass_min_ap),

            # Yield
            "Min Yield": (None if pd.isna(min_yield_pre) else min_yield_pre),
            "Min Yield (payout MIN)": (None if pd.isna(min_yield_min_payout) else min_yield_min_payout),
            "Min Yield (payout MAX)": (None if pd.isna(min_yield_max_payout) else min_yield_max_payout),
            "Min Yield (payout AVG)": (None if pd.isna(min_yield_avg_payout) else min_yield_avg_payout),
        })


resumen_cols = [
    "Cabin","RBD","Tarifas",
    "Min B+F","OD", "B+F aft payout",
    "Max B+F", "Max B+F aft payout",
    "Fareclass",
    "Min Yield",
    "Min Yield (payout MIN)", "Min Yield (payout MAX)",
    "Min Yield (payout AVG)"
]
resumen = pd.DataFrame(filas_resumen, columns=resumen_cols)


# ======= 5) Chequeo de consistencia =======
total_esperado = len(df[(df["RBD_bucket"].notna()) & df[COL_CABIN].isin(buckets_por_cabin.keys())])
print(f"Suma Tarifas en el resumen = {resumen['Tarifas'].sum()} | Filas cubiertas por los buckets = {total_esperado}")

if resumen["Tarifas"].sum() != total_esperado:
    faltantes = df[(df["RBD_bucket"].notna()) & df[COL_CABIN].isin(buckets_por_cabin.keys())].copy()
    faltantes = faltantes[~faltantes["RBD_bucket"].isin(sum(buckets_por_cabin.values(), []))]
    if not faltantes.empty:
        print("⚠️ Hay filas que no cayeron en ningún bucket definido (revisa buckets_por_cabin):")
        print(faltantes[[COL_CABIN, COL_RBD, "RBD_bucket"]].value_counts().head(20).to_string())



# ======= 6) (Opcional) redondeo y export =======
DECIMALS = 2
for c in ["Min B+F", "B+F aft payout",
          "Max B+F", "Max B+F aft payout",      # <-- NUEVAS
          "Min Yield",
          "Min Yield (payout MIN)", "Min Yield (payout MAX)", "Min Yield (payout AVG)"]:
    if c in resumen.columns:
        resumen[c] = pd.to_numeric(resumen[c], errors="coerce").round(DECIMALS)

write_sheet(resumen, excel_path, sheet_name="Resumen por RBD (con payout)", replace=True)
print("Resumen creado en:",  excel_path)



Suma Tarifas en el resumen = 320 | Filas cubiertas por los buckets = 320
Resumen creado en: C:\Users\amvillamizar\Downloads\Avianca\Offline\Documentos\Publicacion tarifas\Pricing_checks\Parametros_Offline\Resumen\140126_PRUEBA_ABS.xlsx


In [ ]:
assert {"Min Yield (payout MIN)", "Min Yield (payout MAX)", "Min Yield (payout AVG)"} <= set(resumen.columns)
print(resumen[["RBD","Min Yield (payout MIN)","Min Yield (payout MAX)","Min Yield (payout AVG)"]].head())

   RBD  Min Yield (payout MIN)  Min Yield (payout MAX)  Min Yield (payout AVG)
0  U-T                     NaN                     NaN                     NaN
1    S                    0.37                    0.37                    0.37
2  W-O                    0.83                    0.83                    0.83
3  L-A                    4.77                    4.77                    4.77
4  V-M                   16.38                   16.38                   16.38


In [ ]:

def _df_to_txt_table(df, max_rows=50, decimals=None):
    """
    Genera tabla Markdown para TXT.
    - Si decimals es None, NO redondea numéricos aquí (útil si ya formateaste).
    - Si decimals es un número, redondea solo columnas numéricas a ese # de decimales.
    """
    tmp = df.copy()

    # Redondeo opcional (por defecto None => sin redondeo aquí)
    if isinstance(decimals, (int, float)):
        for c in tmp.select_dtypes(include="number").columns:
            tmp[c] = tmp[c].round(decimals)

    # Truncar si es muy grande
    truncated = False
    if len(tmp) > max_rows:
        tmp = tmp.iloc[:max_rows].copy()
        truncated = True

    # Reemplazar NaN por vacío
    tmp = tmp.fillna("")

    # Convertir a tabla Markdown estilo GitHub
    table_txt = tabulate(tmp, headers="keys", tablefmt="github", showindex=False)

    if truncated:
        table_txt += f"\n... ({len(df) - max_rows} filas más)"

    return table_txt


# ======== Formateo diferenciado para el TXT ========
sin_decimales = ["Min B+F", "B+F aft payout", "Max B+F", "Max B+F aft payout "]
con_decimales = ["Min Yield", "Min Yield (payout MIN)", "Min Yield (payout MAX)", "Min Yield (payout AVG)"]

# 1) Crea una COPIA para mostrar en TXT (no toques 'resumen' original)
resumen_disp = resumen.copy()

# 2) Asegura numéricos en las columnas objetivo
for c in sin_decimales + con_decimales:
    if c in resumen_disp.columns:
        resumen_disp[c] = pd.to_numeric(resumen_disp[c], errors="coerce")

# 3) Formatea columnas sin decimales como enteros SIN ".0"
for c in sin_decimales:
    if c in resumen_disp.columns:
        # Redondea a 0 y luego convierte a string cuidando NaN
        serie = resumen_disp[c].round(0)
        resumen_disp[c] = serie.apply(lambda x: "" if pd.isna(x) else f"{int(x)}")

# 4) Formatea columnas con 2 decimales como strings "xx.yy"
for c in con_decimales:
    if c in resumen_disp.columns:
        serie = resumen_disp[c].round(2)
        resumen_disp[c] = serie.apply(lambda x: "" if pd.isna(x) else f"{x:.2f}")

# 5) Genera la tabla de texto (NO redondear aquí: decimals=None)
tabla_resumen_txt = _df_to_txt_table(resumen_disp, max_rows=50, decimals=None)

# 6) Sección para el TXT
tabla_resumen_lines = [
    "9. Resumen por RBD\n\n",
    tabla_resumen_txt,
    "\n\n"
]



In [ ]:
resumen.head(5)

,Cabin,RBD,Tarifas,Min B+F,OD,B+F aft payout,Max B+F,Max B+F aft payout,Fareclass,Min Yield,Min Yield (payout MIN),Min Yield (payout MAX),Min Yield (payout AVG)
0,Y,U-T,0,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
1,Y,S,16,171.0,ROCBOG,14.0,290.0,206.00,SLRB1BR9,3.99,0.37,0.37,0.37
2,Y,W-O,96,189.5,ROCBOG,31.5,587.0,491.58,WLRB1BRS,4.42,0.83,0.83,0.83
3,Y,L-A,96,339.5,ROCBOG,181.5,1212.0,1014.99,LLRB1BRS,7.92,4.77,4.77,4.77
4,Y,V-M,80,782.0,ROCBOG,624.0,2597.0,2174.86,VLRB1BRS,18.25,16.38,16.38,16.38


In [ ]:
lines = [] 
lines.append(f"Resumen de chequeos — {fecha}\n\n")
lines.append("="*40 + "\n")

# 4.1) Secciones previas (ya calculadas)
lines += owrt_lines
lines += corners_lines
lines += bundle_lines
lines += tarif_lines
lines += kms_lines
lines += fuel_lines
lines += yield_lines

def build_fareclass_rules_lines(df, writer=None, titulo="6.c Reglas de Fareclass (sólo para opción N)"):
    lines = [f"{titulo}\n"]

    # Asumimos que ya renombraste columnas con 'renombres'
    req_cols = ["Origin", "Destination", "Routing number", "Fareclass"]
    faltan = [c for c in req_cols if c not in df.columns]
    if faltan:
        lines.append("❌ No se puede validar: faltan columnas " + ", ".join(faltan) + "\n\n")
        return lines

    # Uso df (parámetro de entrada)
    df_fc = df[req_cols].copy()
    df_fc["Origin"] = df_fc["Origin"].astype(str).str.strip().str.upper()
    df_fc["Destination"] = df_fc["Destination"].astype(str).str.strip().str.upper()
    df_fc["Fareclass"] = df_fc["Fareclass"].astype(str).str.strip().str.upper()
    df_fc["Routing number"] = df_fc["Routing number"].astype(str).str.strip().str.upper()

    # OD y componentes
    df_fc["OD"] = df_fc["Origin"] + df_fc["Destination"]
    df_fc["Bucket"] = df_fc["Fareclass"].str[3] + df_fc["Fareclass"].str[4]
    df_fc["RBD"] = df_fc["Fareclass"].str[0]
    df_fc["GrupoKey"] = df_fc["OD"] + "_" + df_fc["Routing number"] + "_" + df_fc["Fareclass"].str[1:7]

    # Conjuntos de reglas (estructura)
    REQUIRED_B3 = set(list("YBMQVAEKLOPZW"))
    REQUIRED_B2 = REQUIRED_B3 | {"S"}
    FORBIDDEN_B1 = set(list("YBMH"))

    # Regla U/T: si aparecen, deben estar en B1 y B2, y en ningún otro bucket
    # (esto se valida a nivel OD + RTG, no por GrupoKey)
    UT_RBDS = {"U", "T"}
    UT_REQUIRED_BUCKETS = {"B1", "B2"}

    errores = []

    # =====================================================
    # 1) VALIDACIONES POR GrupoKey (estructura, SIN U/T)
    # =====================================================
    for gkey, grupo in df_fc.groupby("GrupoKey", sort=False):
        any_row = grupo.iloc[0]
        od = any_row["OD"]
        rtg = any_row["Routing number"]

        # Excluir U/T de las validaciones de estructura
        grupo_estructura = grupo[~grupo["RBD"].isin(UT_RBDS)].copy()
        if grupo_estructura.empty:
            # Este GrupoKey sólo tiene U/T (o está vacío): no aplica estructura
            continue

        rbd_by_bucket = (
            grupo_estructura.groupby("Bucket")["RBD"]
                 .apply(lambda s: set(s.dropna().astype(str)))
                 .to_dict()
        )

        # --- Completitud B3 / B2 por estructura (sin U/T) ---
        if "B3" in rbd_by_bucket:
            missing = REQUIRED_B3 - rbd_by_bucket["B3"]
            if missing:
                errores.append((od, rtg, "B3", "faltan " + ",".join(sorted(missing))))

        if "B2" in rbd_by_bucket:
            missing = REQUIRED_B2 - rbd_by_bucket["B2"]
            if missing:
                errores.append((od, rtg, "B2", "faltan " + ",".join(sorted(missing))))

        # --- Prohibición en B1 por estructura (sin U/T) ---
        if "B1" in rbd_by_bucket:
            forb_found = rbd_by_bucket["B1"] & FORBIDDEN_B1
            if forb_found:
                viols = grupo_estructura[
                    (grupo_estructura["Bucket"] == "B1") &
                    (grupo_estructura["RBD"].isin(FORBIDDEN_B1))
                ]
                for _, row in viols.iterrows():
                    errores.append((od, rtg, "B1",
                                    f"RBD prohibido {row['RBD']} en {row['Fareclass']}"))

    # =====================================================
    # 2) REGLA U/T A NIVEL OD + Routing (sin usar GrupoKey)
    # =====================================================
    for (od, rtg), grupo_odrtg in df_fc.groupby(["OD", "Routing number"], sort=False):

        for rbd_ut in UT_RBDS:
            grupo_rbd = grupo_odrtg[grupo_odrtg["RBD"] == rbd_ut]
            if grupo_rbd.empty:
                continue  # si este RBD no aparece para este OD/RTG, nada que validar

            buckets_con_rbd = set(grupo_rbd["Bucket"])

            # 1) Deben estar presentes tanto en B1 como en B2 (al menos una vez en cada bucket)
            missing_buckets = UT_REQUIRED_BUCKETS - buckets_con_rbd
            if missing_buckets:
                detalle = (
                    f"RBD {rbd_ut} debe existir en buckets B1 y B2; "
                    f"falta en: {','.join(sorted(missing_buckets))}"
                )
                errores.append((od, rtg, "/".join(sorted(buckets_con_rbd)), detalle))

            # 2) No deben aparecer en otros buckets distintos de B1/B2
            extra_buckets = buckets_con_rbd - UT_REQUIRED_BUCKETS
            if extra_buckets:
                viols_extra = grupo_rbd[grupo_rbd["Bucket"].isin(extra_buckets)]
                for _, row in viols_extra.iterrows():
                    errores.append(
                        (
                            od,
                            rtg,
                            row["Bucket"],
                            f"RBD {rbd_ut} sólo permitido en B1 y B2 "
                            f"(encontrado en {row['Bucket']} - Fareclass {row['Fareclass']})"
                        )
                    )

    # =====================================================
    # 3) SALIDA
    # =====================================================
    if not errores:
        lines.append("✅ Todas las reglas de Fareclass se cumplen.\n\n")
        return lines

    # En TXT → solo primeros 10
    lines.append(f"❌ Se encontraron {len(errores)} inconsistencias en Fareclass. Mostrando primeras 10:\n")
    for e in errores[:10]:
        lines.append(f"- OD={e[0]}, RTG={e[1]}, Bucket={e[2]} → {e[3]}\n")
    lines.append("\n")

    # En Excel → exporta todas si hay writer
    if writer is not None:
        df_err = pd.DataFrame(errores, columns=["OD", "Routing number", "Bucket", "Detalle"])
        df_err.to_excel(writer, sheet_name="Errores_Fareclass", index=False)

    return lines




op = elegir_opcion_action_code()   # "N" | "C" | "I/R" | None
if op is None:
    sel_lines = ["6.b Validación tipo de request\n", "❌ Operación cancelada por el usuario.\n\n"]
else:
    sel_lines = build_selection_validation_lines(
        df_yield_ap, COL, op,
        titulo="6.b Validación del tipo de request (según selección del usuario)"
    )
lines += sel_lines

# ========= 4) Validación de reglas de Fareclass cuando opción = "N" =========
if op == "N":
    # Usa el DF ya renombrado (con 'Origin', 'Destination', 'Routing number', 'Fareclass')
    lines += build_fareclass_rules_lines(df_yield_ap, writer=None)

baseprice_lines = build_baseprice_check_lines(
    df_yield_ap,
    col_price="Tarifa_inicial_USD",
    col_owrt="OW/RT",
    col_od="OD",
    col_fc="Fareclass",
    titulo="7. Validación Base price minimo por OW/RT"
)
lines += baseprice_lines
owrt_tarifa_lines = build_owrt_tarifa_check_lines(
    df_filtrado_final,
    col_od="OD",
    col_rtg="Routing number",
    col_rbd="RBD",
    col_bundle="Bundle",
    col_flag="OW/RT",
    col_tarifa="Base_mas_Fuel",
    col_fc="Fareclass",
    titulo="8. Validación tarifa OW > RT"
)
lines += owrt_tarifa_lines

# 4.3) Tabla resumen (al final, como ya hacías)
lines += tabla_resumen_lines

# 4.4) Escribir archivo
with open(reporte_path, "w", encoding="utf-8") as f:
    f.write("".join(lines))

print("✅ Resumen TXT guardado en:", reporte_path)

✅ Resumen TXT guardado en: C:\Users\amvillamizar\Downloads\Avianca\Offline\Documentos\Publicacion tarifas\Pricing_checks\Parametros_Offline\Resumen\140126_PRUEBA_ABS.txt


In [ ]:
with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    # otras hojas...
    if op == "N":
        lines += build_fareclass_rules_lines(df_yield_ap, writer=writer)

In [ ]:
with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    owrt_tarifa_lines = build_owrt_tarifa_check_lines(
        df_yield_ap,
        col_od="OD",
        col_rtg="Routing number",
        col_rbd="RBD",
        col_bundle="Bundle",
        col_flag="OW/RT",
        col_tarifa="Base_mas_Fuel",
        col_fc="Fareclass",
        writer=writer)
    lines += owrt_tarifa_lines

In [ ]:
df_yield_ap.to_excel("df_tarifas.xlsx", index=False)